In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
from matplotlib_venn import venn2, venn2_circles
import argparse
from create_reference_from_tsv_and_pepxml import ReferenceWriter
from sensitivity_calculator import SensitivityAndSpecificity
from create_PSM_df import PSM_FDR
from Bio import SeqIO
from collections import defaultdict
from Bio import pairwise2 as pw2
import wget

In [2]:
# load taxon graph
import sys  
sys.path.insert(0, '/home/jules/tax2proteome_projects/tax2proteome/')
from TaxonGraph import TaxonGraph
taxon_graph = TaxonGraph()
taxon_graph.create_graph("/home/jules/Documents/Metaproteomics/databases/databases_tax2proteome/taxdump.tar.gz")

In [3]:
# Files
#new_reduced: ohne +1 charge specta
uniprot_nr_species_tsv_with_wrong_error = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/uniprot/x_tandem_tsv/Run1_U1_2000ng_uniprot_species_nr.t.xml_new_reduced.tsv"


reference_tsv_with_kleiner_db = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_kleiner_db/Run1_U1_2000ng.t.xml_reduced.tsv"
reference_tsv_with_aradiopsis = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_kleiner_reference_aradiopsis/Run1_U1_2000ng_kleiner_aradiopsis.t.xml_reduced.tsv"
reference_tsv_with_ara_yeast_human = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_kleiner_ara_yeast_human/Run1_U1_2000ng.t.xml_new_reduced.tsv"
path_to_bachelor_results = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/"
path_to_uniprot_reanalysis = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_reanalysis_uniprot"
path_to_tanca_results_bachelor = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/9MM_FASP/x_tandem_tsv"
path_to_kleiner_species_nr_0_9_identity = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/uniprot_species_sequences_0_9_identity/Run1_U1_2000ng_uniprot_species_0_9_identity.t.xml.tsv"
path_to_kleiner_species_nr_0_9_identity_reduced = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/uniprot_species_sequences_0_9_identity/Run1_U1_2000ng_uniprot_species_0_9_identity.t.xml_new_reduced.tsv"
def get_path_to_refernce_analysis_results(fdr):
    return f"/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_kleiner_db/Run1_U1_2000ng.t.xml_reduced.tsv_{fdr}_sensitivity_new"

def get_path_to_refernce_analysis_results_with_aradiopsis(fdr):
    return f"/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_kleiner_reference_aradiopsis/Run1_U1_2000ng_kleiner_aradiopsis.t.xml_reduced.tsv_{fdr}_sensitivity_new"



uniprot_reanalysis = {'species': path_to_uniprot_reanalysis + "/species/Run1_U1_2000ng_uniprot_species.t.xml_new_reduced.tsv",
                      'species_nr': path_to_uniprot_reanalysis + "/species_nr/Run1_U1_2000ng_uniprot_species_nr.t.xml_reduced.tsv",
                      "species_species": path_to_uniprot_reanalysis + "/species_species/Run1_U1_2000ng_uniprot_species_species.t.xml_reduced.tsv",
                      'genus_nr': path_to_uniprot_reanalysis + "/genus_nr/Run1_U1_2000ng_uniprot_genus_nr.t.xml_reduced.tsv"
                     }
    
uniprot_nr_reduced_tsv = {
    'subspecies': path_to_bachelor_results + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_subspecies.t.xml_new_reduced.tsv",
    'species': path_to_bachelor_results + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_species_nr.t.xml_new_reduced.tsv",
    'genus': path_to_bachelor_results + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_genus_nr.t.xml_new_reduced.tsv",
    'family': path_to_bachelor_results + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_family_nr.t.xml_new_reduced.tsv",
}
    
uniprot_reduced_tsv = {
    'subspecies': path_to_bachelor_results + "/ncbi_kleiner/x_tandem_tsv/Run1_U1_2000ng_ncbi_kleiner_subspecies.t.xml_new_reduced.tsv",
    'species': path_to_bachelor_results + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_species.t.xml_new_reduced.tsv",
    'genus': path_to_bachelor_results + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_genus.t.xml_new_reduced.tsv"
}

uniprot_species_reduced_tsv =  {
    'subspecies': path_to_uniprot_reanalysis + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_subspecies.t.xml_new_reduced.tsv",
    'species': path_to_bachelor_results + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_species_species.t.xml_new_reduced.tsv",
    'genus':  path_to_bachelor_results + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_genus_species.t.xml_new_reduced.tsv"
}

ncbi_reduced_tsv= {
    'subspecies': path_to_bachelor_results + "/ncbi_kleiner/x_tandem_tsv/Run1_U1_2000ng_ncbi_kleiner_subspecies.t.xml_new_reduced.tsv",
    'species': path_to_bachelor_results + "/ncbi_kleiner/x_tandem_tsv/Run1_U1_2000ng_ncbi_kleiner_species.t.xml_new_reduced.tsv",
    'genus': path_to_bachelor_results + "/ncbi_kleiner/x_tandem_tsv/Run1_U1_2000ng_ncbi_kleiner_genus.t.xml_new_reduced.tsv"
}

ncbi_species_reduced_tsv = {
    'species': path_to_bachelor_results + "/ncbi_kleiner/x_tandem_tsv/Run1_U1_2000ng_ncbi_kleiner_species_species.t.xml_new_reduced.tsv",
    'genus': path_to_bachelor_results + "/ncbi_kleiner/x_tandem_tsv/Run1_U1_2000ng_ncbi_kleiner_genus_species.t.xml_new_reduced.tsv"
}
swissprot_reduced_tsv = {
    'species': path_to_bachelor_results + "/swissprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_swissprot_species.t.xml_new_reduced.tsv",
    'genus': path_to_bachelor_results + "/swissprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_swissprot_genus.t.xml_new_reduced.tsv",
    'family': path_to_bachelor_results + "/swissprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_swissprot_family.t.xml_new_reduced.tsv",
    'order': path_to_bachelor_results + "/swissprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_swissprot_order.t.xml_new_reduced.tsv"
}

tanca_ncbi_tsv = {
    'species': path_to_tanca_results_bachelor + "/9MM_FASP_ncbi_Tanca_species.t.xml_new_reduced.tsv",
    'genus': path_to_tanca_results_bachelor + "/9MM_FASP_ncbi_tanca_genus.t.xml_new_reduced.tsv",
    'family': path_to_tanca_results_bachelor + "/9MM_FASP_ncbi_tanca_family.t.xml_new_reduced.tsv",
}

tanca_uniprot_tsv = {
    'species': path_to_tanca_results_bachelor + "/9MM_FASP_uniprot_Tanca_species_nr.t.xml_new_reduced.tsv",
    'genus': path_to_tanca_results_bachelor + "/9MM_FASP_uniprot_Tanca_genus.t.xml_new_reduced.tsv",
    'family': path_to_tanca_results_bachelor + "/9MM_FASP_uniprot_Tanca_family_nr.t.xml_new_reduced.tsv",
    'order': path_to_tanca_results_bachelor + "/9MM_FASP_uniprot_Tanca_order_nr.t.xml_new_reduced.tsv"
}

tanca_swissprot_tsv = {
    'species': path_to_tanca_results_bachelor + "/9MM_FASP_swissprot_tanca_species.t.xml_new_reduced.tsv",
    'genus': path_to_tanca_results_bachelor + "/9MM_FASP_swissprot_tanca_genus.t.xml_new_reduced.tsv",
    'family': path_to_tanca_results_bachelor + "/9MM_FASP_swissprot_tanca_family.t.xml_new_reduced.tsv",
    'order': path_to_tanca_results_bachelor + "/9MM_FASP_swissprot_tanca_order.t.xml_new_reduced.tsv"
}

def get_dict_databases_to_size_and_result_file(fdr):
    dict_databases_to_size_and_result_file={'reference': (123088, f"{reference_tsv_with_kleiner_db}_{fdr}_sensitivity_new"),
                                        "reference_with_aradiopsis": (138980, f"{reference_tsv_with_aradiopsis}_{fdr}_sensitivity_new"),
                                        
                                        'ncbi_subspecies': (300000, f"{ncbi_reduced_tsv['subspecies']}_{fdr}_sensitivity_new"),
                                        'ncbi_species': (8702135, f"{ncbi_reduced_tsv['species']}_{fdr}_sensitivity_new"),
                                        "ncbi_genus":(27804893, f"{ncbi_reduced_tsv['genus']}_{fdr}_sensitivity_new"),
                                        
                                        "uniprot_subspecies": (4683371, f"{uniprot_reduced_tsv['subspecies']}_{fdr}_sensitivity_new"),
                                        "uniprot_species": (4683371, f"{uniprot_reduced_tsv['species']}_{fdr}_sensitivity_new"),
                                        "uniprot_species_species": (2093157, f"{uniprot_species_reduced_tsv['species']}_{fdr}_sensitivity_new"),
                                        "uniprot_subspecies_nr": (200000, f"{uniprot_nr_reduced_tsv['subspecies']}_{fdr}_sensitivity_new"),
                                        "uniprot_species_nr": (2991727, f"{uniprot_nr_reduced_tsv['species']}_{fdr}_sensitivity_new"),
                                        "uniprot_genus": (18352148,  f"{uniprot_reduced_tsv['genus']}_{fdr}_sensitivity_new"),
                                        "uniprot_genus_species": (13068285,  f"{uniprot_species_reduced_tsv['genus']}_{fdr}_sensitivity_new"),
                                        "uniprot_genus_nr": (13210287,  f"{uniprot_nr_reduced_tsv['genus']}_{fdr}_sensitivity_new"),
                                        "uniprot_family_nr": (22509624, f"{uniprot_nr_reduced_tsv['family']}_{fdr}_sensitivity_new"),
                                        
                                        "swissprot_species": (58505, f"{swissprot_reduced_tsv['species']}_{fdr}_sensitivity_new"),
                                        "swissprot_genus": (88164, f"{swissprot_reduced_tsv['genus']}_{fdr}_sensitivity_new" ),
                                        "swissprot_family": (124044, f"{swissprot_reduced_tsv['family']}_{fdr}_sensitivity_new"),
                                        "swissprot_order": (181725, f"{swissprot_reduced_tsv['order']}_{fdr}_sensitivity_new")
                                       }
    return dict_databases_to_size_and_result_file

dict_databases_to_size_and_result_file = get_dict_databases_to_size_and_result_file(0.05)
uniprot_dict =dict(filter(lambda item: item[0] in ['uniprot_subspecies', 'uniprot_species', 'uniprot_genus'], dict_databases_to_size_and_result_file.items()))
uniprot_nr_dict =dict(filter(lambda item: '_nr' in item[0], dict_databases_to_size_and_result_file.items()))
uniprot_species_dict=dict(filter(lambda item: item[0] in ['uniprot_species_species', 'uniprot_genus_species'], dict_databases_to_size_and_result_file.items()))
ncbi_dict =dict(filter(lambda item: 'ncbi' in item[0], dict_databases_to_size_and_result_file.items()))
swissprot_dict=dict(filter(lambda item: 'swiss' in item[0], dict_databases_to_size_and_result_file.items()))

In [4]:
def get_all_rows_without_charge1(title_col):
    return [title.split('.')[-1] != '' for title in title_col]

def remove_spectra_of_charge_one_from_reduced_tsv(reduced_tsv):
    return reduced_tsv[get_all_rows_without_charge1(reduced_tsv.Title)]

def read_all_results_into_dict(file):
    result_dict= {}
    with open(file, 'r') as inp:
        firstline=inp.readline()
        db_names = [name.strip() for name in firstline.strip().split('\t')[1:]]
        for db in db_names:
            result_dict[db] = {}
        for line in inp.readlines():
            result_name = line.split('\t')[0]
            result_fields = [name.strip() for name in line.split('\t')[1:]]
            for i, db in enumerate(db_names):
                result_dict[db][result_name]=result_fields[i]
    return result_dict

def read_analysis_results_into_dict(file):
    result_dict= {}
    with open(file, 'r') as inp:
        for line in inp.readlines():
            result_dict[line.split('\t')[0]] = line.split('\t')[1].strip()
   #print(result_dict)
    return result_dict

def read_TP(file):
    TP = int(read_analysis_results_into_dict(file)['TP:'])
    return TP

def read_TN(file):
    TN = int(read_analysis_results_into_dict(file)['TN:'])
    return TN

def read_FP(file):
    TP = int(read_analysis_results_into_dict(file)['FP:'])
    return TP

def read_FN(file):
    TN = int(read_analysis_results_into_dict(file)['FN:'])
    return TN

def read_sensitivity(file):
    sen = float(read_analysis_results_into_dict(file)['sensitivity:'])
    return sen
 
def read_specificity(file):
    spe = float(read_analysis_results_into_dict(file)['specificity:'])
    return spe


def read_value_from_result_file(file, database, level, column_of_interest):
    df = pd.read_csv(file, sep='\t')
    value = df.loc[(df['level'] == level) & (df['database'] == database)][column_of_interest]
    return value.values[0]


In [5]:
def get_score_results(filtered_dict):   
    db_size = []
    score_border = []
    result_dict = {}
    for file_name, db_size_file_tuple in filtered_dict.items():
        result_dict[file_name] = {'db_size': db_size_file_tuple[0]/100000, 'score_border': read_last_score(db_size_file_tuple[1])}

    for k, v in result_dict.items():
        db_size.append(v['db_size'])
        score_border.append(v['score_border'])
    return result_dict, db_size, score_border

def get_specificity_results(filtered_dict):   
    db_size = []
    score_border = []
    result_dict = {}
    for file_name, db_size_file_tuple in filtered_dict.items():
        result_dict[file_name] = {'db_size': db_size_file_tuple[0]/100000, 'specificity': read_specificity(db_size_file_tuple[1])}
    for k, v in result_dict.items():
        db_size.append(v['db_size'])
        score_border.append(v['specificity'])
    return result_dict, db_size, score_border

In [6]:
def get_decoy_rows_from_decoy_column(decoy_column):
    return [True if d=={True} else False for d in decoy_column]

def get_tax_rows(tax_column, taxid):
    return [True if tax_set == {taxid} else False for tax_set in tax_column]

def get_decoy_rows_from_protein_column(protein_column):
    return [True if 'REVERSED' in p else False for p in protein_column]


def get_hit_rows(protein_column):
    return [False if 'REVERSED' in p else True for p in protein_column]

def get_decoy_rows2(decoy_column):
    return [True if d_set in [{True, False}, {True}] else False for d_set in decoy_column]

def get_hit_rows2(decoy_column):
    return [True if d_set in  [{True, False}, {False}] else False for d_set in decoy_column]

def get_not_crap_rows(column_name):
    return [True if t_set != {"CRAP"} else False for t_set in column_name]

def get_taxa_rows(column_name, taxID):
    return [True if taxID in t_set else False for t_set in column_name]

def get_spectra_rows(spectra_column, spectra_set):
    return [True if spectrum in spectra_set else False for spectrum in spectra_column]
def get_not_spectra_rows(spectra_column, spectra_set):
    return [True if spectrum not in spectra_set else False for spectrum in spectra_column]

def get_acc_rows(acc_column, acc_set1, all_acc_contained=True):
    """
    returns rows, where ALL acc in row also in acc_set1
    """
    if all_acc_contained:
        return [True if len(acc_set2.difference(acc_set1)) == 0 else False for acc_set2 in acc_column]
    else:
        return [True if len(acc_set2.intersection(acc_set1)) > 0 else False for acc_set2 in acc_column]

def get_not_acc_rows(acc_column, acc_set1, all_acc_contained=True):
    if all_acc_contained:
        return [False if len(acc_set2.difference(acc_set1)) == 0  else True for acc_set2 in acc_column]
    else:
        return [False if len(acc_set2.intersection(acc_set1)) > 0  else True for acc_set2 in acc_column]

In [7]:
def read_result_df(f):
    df =  ReferenceWriter.read_csv_with_generic_function(f,['Hyperscore', 'taxID', 'decoy'])
    return df[['Title', 'Peptide', 'Hyperscore', 'taxID', 'decoy']]
    
def get_psm_and_df_in_fdr(file, fdr, remove_one_charged_spectra=False, columns=None):
    cs = ['Protein', 'Hyperscore', 'decoy', 'taxID']
    if columns:
        cs = cs + columns
    reduced_df = ReferenceWriter.read_csv_with_generic_function(file, cs, remove_one_charged_spectra)
    fdr_pos_result, number_psm_result, number_decoy_result, double_spectra_result, score_last_item_result = PSM_FDR.determine_FDR_position(reduced_df, fdr)
    return number_psm_result, reduced_df[0:fdr_pos_result]

def get_df_in_fdr_without_decoy(file, fdr, remove_one_charged_spectra=True, columns=None):
    df = get_psm_and_df_in_fdr(file, fdr, remove_one_charged_spectra, columns)[1]
    df = df[get_hit_rows2(df.decoy)]
    return df

In [8]:
# Histogramm
def create_df_for_df_with_sep_decoy(df_reduced_tsv):
    """
    decoy column values of reduced_tsv file changed to 'PSM' for  {False} and 'DECOY' for {TRUE}
    {True, False} values doubled, one as 'PSM' one as 'DECOY'
    :return:  Title(=spectrum ID) | Protein set {acc1, acc2} | Hyperscore (float) | decoy (PSM or DECOY)
    """
    def get_true_false(decoy_column):
        l = [True if decoy == {True, False} else False for decoy in decoy_column]
        return l
    # reduce df to columns 'Title', 'Protein', 'Hyperscore', 'decoy'
   # df_reduced_tsv = df_reduced_tsv[['Title', 'Protein', 'Hyperscore', 'decoy']]
    # df only with rows with value {True, False} in column "decoy"
    df_true_false = df_reduced_tsv[get_true_false(df_reduced_tsv.decoy)]
    # df only with rows with value {True, False} in column "decoy"  copyied
    df_true = df_true_false.copy(deep=True)
    # df value {True, False} in column "decoy" changed to {True}
    df_true.decoy=[{True} for v in df_true.decoy]
    # copied original df with columns 'Title', 'Protein', 'Hyperscore', 'decoy'
    df_with_sep_decoy= df_reduced_tsv.copy(deep=True)
    # decoy column value {False, True} changed to False
    df_with_sep_decoy.decoy = [{False} if v == {False, True} else v for v in df_with_sep_decoy.decoy ]
    # df_true rows appended, so decoy entries with {False True} doubled, one with {true}, once with {False}}
    df_with_sep_decoy = df_with_sep_decoy.append(df_true, ignore_index=True)
    # replace decoy value {True} by DECOY {False} by 'PSM'
    df_with_sep_decoy.decoy=['DECOY' if v == {True} else 'PSM' for v in df_with_sep_decoy.decoy]
    return df_with_sep_decoy


In [9]:

def create_all_level_df_for_histogram_per_db(database_reduced_tsv, db_levels):
    """
    database_reduced_tsv e.g. uniprot_nr_reduced_tsv (dict) or list of paths to reduced tsvs
    db_levels: e.g.[species, genus...] or [db_1, db_2]
    """
    
    df_reduced_tsv_list = []
    result_df_list = []
    
    if type(database_reduced_tsv)==list:
        for reduced_tsv in database_reduced_tsv:
            df_reduced_tsv_list.append(ReferenceWriter.read_csv_with_generic_function(reduced_tsv,['Hyperscore',  'decoy'])[['Title', 'Hyperscore',  'decoy']])
        for i, df_reduced_tsv in enumerate(df_reduced_tsv_list):
            df_level = create_df_for_df_with_sep_decoy(df_reduced_tsv)
            df_level['database'] = db_levels[i]
            result_df_list.append(df_level)            
    else:   
        for level in db_levels:
            df_reduced_tsv_list.append(ReferenceWriter.read_csv_with_generic_function(database_reduced_tsv[level],['Hyperscore',  'decoy'])[['Title', 'Hyperscore',  'decoy']])
        for i, df_reduced_tsv in enumerate(df_reduced_tsv_list):
            df_level = create_df_for_df_with_sep_decoy(df_reduced_tsv)
            # change subspecies to strain
            if db_levels[i] == 'subspecies':
                level = 'strain'
            else:
                level = db_levels[i]
            df_level['level'] = level
            result_df_list.append(df_level)      
    df_all_level = pd.concat(result_df_list)
    return df_all_level

def get_all_fdr_borders_per_db(db_dict, db_names):
    border_list = []
    for name in (db_names):
        fdr_border_level = float(read_last_score(db_dict[name][1]))
        border_list.append(fdr_border_level)
    return border_list

In [10]:
# create df sensitivity analysis and save
def create_sens_dict(file, db, level):
    d = read_analysis_results_into_dict(file)
    d['database'] = db
    d['level'] = level
    return d

def create_all_analysis_result_df(fdr, tanca=False):
    dict_databases_to_size_and_result_file = get_dict_databases_to_size_and_result_file(fdr)
    d_list = []
    for level in ['subspecies', 'species', 'genus', 'family', 'order']:
        try:
            d = create_sens_dict(dict_databases_to_size_and_result_file[f'uniprot_{level}_nr'][1], 'uniprot_nr', level)
            d_list.append(d)
        except FileNotFoundError:
            pass
        except KeyError:
            pass
        try:
            d = create_sens_dict(dict_databases_to_size_and_result_file[f'uniprot_{level}'][1], 'uniprot', level)
            d_list.append(d)
        except FileNotFoundError:
            pass
        except KeyError:
            pass
        try:
            d = create_sens_dict(dict_databases_to_size_and_result_file[f'ncbi_{level}'][1], 'ncbi', level)
            d_list.append(d)
        except FileNotFoundError:
            pass
        except KeyError:
            pass
        try:
            d = create_sens_dict(dict_databases_to_size_and_result_file[f'swissprot_{level}'][1], 'swissprot', level)
            d_list.append(d)
        except FileNotFoundError:
            pass
        except KeyError:
            pass
    ref_list= []
    ref_d =create_sens_dict(get_path_to_refernce_analysis_results(fdr), 'reference', '-')
    ref_list.insert(0, ref_d)
    ref_d =create_sens_dict(get_path_to_refernce_analysis_results_with_aradiopsis(fdr), 'reference with aradiopsis seq', '-')
    ref_list.append(ref_d)

    full_df = pd.DataFrame(d_list)
    full_df = full_df.sort_values(by=['database'], ignore_index=True)
    full_df = pd.concat([pd.DataFrame(ref_list), full_df],  ignore_index=True)
    full_df.columns = [head[0:-1] if head.endswith(':') else head for head in list(full_df.columns) ]
    full_df = full_df.rename(columns={'Number of spectra identified in result but not in reference': "nb additional identified spectra", 
                            "Number of spectra identified in result": "nb identified spectra",
                           "Number of PSMs in result": "nb PSMs",
                           "Hyperscore of last item in FDR boundaries": "lowest score in FDR"})
    df = full_df[['database', 'level', 'specificity', 'sensitivity', 'TP', 'FP', 'TN', 'FN', "nb additional identified spectra",
                 "nb identified spectra", "nb PSMs", "lowest score in FDR"]]
    return df


In [11]:
# database-size vs. PSMs
def get_psm_per_db_from_sens_analysis(df, db):
    PSM_identified = []
    if db == 'uniprot':
        PSM_identified.append(int(df.loc[df['database'] == 'uniprot' ].loc[df['level'] == 'subspecies']['nb PSMs']))
        PSM_identified.append(int(df.loc[df['database'] == 'uniprot_nr' ].loc[df['level'] == 'species']['nb PSMs']))
        PSM_identified.append(int(df.loc[df['database'] == 'uniprot_nr' ].loc[df['level'] == 'genus']['nb PSMs']))
        PSM_identified.append(int(df.loc[df['database'] == 'uniprot_nr' ].loc[df['level'] == 'family']['nb PSMs']))
        
    if db == 'ncbi':
        PSM_identified.append(int(df.loc[df['database'] == 'ncbi' ].loc[df['level'] == 'subspecies']['nb PSMs']))
        PSM_identified.append(int(df.loc[df['database'] == 'ncbi' ].loc[df['level'] == 'species']['nb PSMs']))
        PSM_identified.append(int(df.loc[df['database'] == 'ncbi' ].loc[df['level'] == 'genus']['nb PSMs']))
        
    if db == 'swissprot':        
        PSM_identified.append(int(df.loc[df['database'] == 'swissprot' ].loc[df['level'] == 'species']['nb PSMs']))
        PSM_identified.append(int(df.loc[df['database'] == 'swissprot' ].loc[df['level'] == 'genus']['nb PSMs']))
        PSM_identified.append(int(df.loc[df['database'] == 'swissprot' ].loc[df['level'] == 'family']['nb PSMs']))
        PSM_identified.append(int(df.loc[df['database'] == 'swissprot' ].loc[df['level'] == 'order']['nb PSMs']))
    return PSM_identified

def create_pd_df_for_psm_plot(PSM_identified_uniprot_percentage, PSM_identified_ncbi_percentage, uniprot_database_sizes, ncbi_database_sizes, PSM_identified_swissprot_percentage=None, swissprot_database_sizes=None):
    database = ['UniProtKB' for e in PSM_identified_uniprot_percentage]
    database.extend(['NCBI' for e in PSM_identified_ncbi_percentage])
    if PSM_identified_swissprot_percentage:
        database.extend(['Swissprot' for e in PSM_identified_swissprot_percentage])
    psms = PSM_identified_uniprot_percentage + PSM_identified_ncbi_percentage 
    if PSM_identified_swissprot_percentage:
        psms = psms + PSM_identified_swissprot_percentage
    dataset = ['kleiner' for e in  database]
    level = ['strain', 'species', 'genus', 'family', 'strain', 'species', 'genus']
    if PSM_identified_swissprot_percentage:
        level = level + ['species', 'genus', 'family', 'order']

    database_size = uniprot_database_sizes + ncbi_database_sizes
    if swissprot_database_sizes:
        database_size = database_size + swissprot_database_sizes
    DatabaseSize = pd.DataFrame(list(zip(database_size, level, psms, database, dataset)), columns =['# fasta (M)', 'level', '% PSMs identified', 'database', 'dataset'])
    return DatabaseSize

In [12]:
# identified spectra in all levels together: 36,84%, deutlich abweichender Wert von Bachelorarbeit 26%:
def get_decoy_rows_from_decoy_column_from_merged_df(decoy_column):
    result=[]
    for d in decoy_column:
        # nan 
        if type(d) != set:
            result.append(True)
        elif  d=={True}:
            result.append(True)
        elif  d=={False}:
            result.append(False)
        elif  d=={True, False}:
            result.append(False)
        else:
            print(d)
    return result

def get_merged_df_of_all_levels(reduced_tsv_file_dict, fdr, levels):
    dfs_in_fdr = []
    decoy_columns = []
    for level in levels:
        number_psm_result, reduced_df_in_fdr = get_psm_and_df_in_fdr(reduced_tsv_file_dict[level], fdr)[1]
        reduced_df_in_fdr = reduced_df_in_fdr.rename(columns={'decoy': f'decoy_{level}'})
        decoy_columns.append(f'decoy_{level}')
        dfs_in_fdr.append(reduced_df_in_fdr)
    
    merged_df = pd.DataFrame(columns=["Title"])
    for reduced_df_in_fdr in dfs_in_fdr:
        merged_df = pd.merge(merged_df, reduced_df_in_fdr, how="outer", on='Title')

    decoy_lists_per_level = []
    for decoy_column_name in decoy_columns:
        decoy_list_per_level = get_decoy_rows_from_decoy_column_from_merged_df(merged_df[decoy_column_name])
        decoy_lists_per_level.append(decoy_list_per_level)
    
    final_decoy_list= []
    for decoy_tuple in zip(*decoy_lists_per_level):
        if False in decoy_tuple:
            final_decoy_list.append(True)
        else:
            final_decoy_list.append(False)
            
    identified_spectra_of_all_levels_together = len(set(merged_df[final_decoy_list]['Title']))
    return merged_df, final_decoy_list, identified_spectra_of_all_levels_together



In [13]:
def count_spectra_per_taxon(reduced_df_in_fdr, level, taxon=None, taxa_list=None):
    spectra = set()
    for spectrum_ID, taxa_set in zip(list(reduced_df_in_fdr['Title']), list(reduced_df_in_fdr[f'taxID_{level}'])):
        if taxon:
            if taxon in taxa_set:
                spectra.add(spectrum_ID)
        elif taxa_list:
            for taxon in taxa_list:
                if taxon in taxa_set:
                    spectra.add(spectrum_ID)
    return (spectra)

def get_percentage_psm_per_taxon(obj, taxID_to_spectra_dict, taxon_str, level):
    final_taxa_set= set()
    psm_percentage = {}
    for taxon in taxon_str.split(', '):
        taxon = int(taxon) if taxon not in ['viruses', 'CRAP','DECOY'] else taxon
        taxon_of_level = obj.taxon_graph.find_level_up(taxon, level) if level != 'subspecies' else taxon
        final_taxa_set=final_taxa_set.union(taxID_to_spectra_dict[taxon_of_level])
    psm_percentage = len(final_taxa_set)/obj.psm_count*100
    return psm_percentage

def get_sized_down_to_100_percentage(percent, sum_percent):
    return percent/sum_percent*100

In [14]:
def calculate_mean_deviation(path_to_reduced_df, path_to_bachelor_result, level, fdr):
    # Mean deviation between percentage identified PSMs per species and protein composition.
    # Mean deviation = 1/n  ∗ ∑ |% PSMspecies i − % protein species i |
    perc_per_taxa_df = get_perc_per_taxa_df(path_to_reduced_df, path_to_bachelor_result, level, fdr)
    sum_mean = 0
    for row in perc_per_taxa_df.values.tolist():
        sum_mean += abs(row[1]-row[2])
    result = sum_mean/len(perc_per_taxa_df)
    return result

def create_mean_deviation_df(input_files):
    # level    database dataset  mean deviation
    fdr = 0.1
    dict_list = []
    for files in input_files['swissprot']:       
        mean_deviation = calculate_mean_deviation(files[0], files[1], files[2], fdr)
        mean_dict = {'level': files[2], 'database': 'SwissProt', 'dataset': '29MM', 'mean deviation': mean_deviation}
        dict_list.append(mean_dict)
    for files in input_files['ncbi']:       
        mean_deviation = calculate_mean_deviation(files[0], files[1], files[2], fdr)
        mean_dict = {'level': files[2], 'database': 'NCBI', 'dataset': '29MM', 'mean deviation': mean_deviation}
        dict_list.append(mean_dict)
    for files in input_files['uniprot']:       
        mean_deviation = calculate_mean_deviation(files[0], files[1], files[2], fdr)
        mean_dict = {'level': files[2], 'database': 'UniProtKB', 'dataset': '29MM', 'mean deviation': mean_deviation}
        dict_list.append(mean_dict)
    mean_deviation_df = pd.DataFrame(dict_list)
    return mean_deviation_df

In [15]:
import importlib
import number_of_psm_per_species 
from number_of_psm_per_species import PsmNumberPerTaxIDs
importlib.reload(number_of_psm_per_species)
def get_perc_per_taxa_df(path_to_reduced_df, path_to_bachelor_result, level, fdr):
    obj = PsmNumberPerTaxIDs('kleiner', path_to_reduced_df, level, fdr)    
    taxID_to_spectra_dict = obj.count_row_by_row()
    df = pd.read_csv(path_to_bachelor_result, sep='\t')
   # taxon_to_db_percentage = {t:db for t, db in zip(list(df['taxon(s)']), list(df['% database']))}
    df['% PSM_not_adapted'] = df.apply(lambda row: get_percentage_psm_per_taxon(obj, taxID_to_spectra_dict, row['taxon(s)'], level), axis=1)
    sum_percent = df[~df['% PSM_not_adapted'].isin(['CRAP','DECOY', 'virus'])]['% PSM_not_adapted'].sum()
    df['% PSM'] = df.apply(lambda row: get_sized_down_to_100_percentage(row['% PSM_not_adapted'], sum_percent), axis=1)
    # print(df)
    df = df[["taxon(s)", "% PSM", "% protein", "% database"]]
    return df

In [16]:
def get_percentage_identified_spectra(db, level, fdr):
    if 'uniprot' in db.lower():
        db= 'uniprot'
    if 'ncbi' in db.lower():
        db='ncbi'
    if 'swiss' in db.lower():
        db='swissprot'
    nb_all_spectra = 88279
    if db == 'uniprot':
        if level == 'united':
            return get_merged_df_of_all_levels(uniprot_nr_reduced_tsv, fdr, ['subspecies', 'species', 'genus', 'family'])[2]/nb_all_spectra * 100
        else:
            psm, df_in_fdr = get_psm_and_df_in_fdr(uniprot_nr_reduced_tsv[level], fdr)
    if db == 'ncbi':
        if level == 'united':
            return get_merged_df_of_all_levels(ncbi_reduced_tsv, fdr, ['subspecies', 'species', 'genus'])[2]/nb_all_spectra * 100
        else:
             psm, df_in_fdr = get_psm_and_df_in_fdr(ncbi_reduced_tsv[level], fdr)
    if db == 'swissprot':
        if level == 'united':
            return get_merged_df_of_all_levels(swissprot_reduced_tsv, fdr, ['species', 'genus', 'family', "order"])[2]/nb_all_spectra * 100
        else:
            psm, df_in_fdr = get_psm_and_df_in_fdr(swissprot_reduced_tsv[level], fdr)
    if db == 'reference':
        psm, df_in_fdr = get_psm_and_df_in_fdr(reference_tsv_with_kleiner_db, fdr)   
    df_no_decoys = df_in_fdr[get_hit_rows2(df_in_fdr.decoy)]
    return len(set(df_no_decoys['Title']))/nb_all_spectra * 100

In [17]:
## PLOTS FUNCTIONS
def create_histograms_in_columns_per_db(df_all_level, border_list, output, title, fdr, c='level'):
    """
    multiple histogramms in one row
    """
    sns.set_style("whitegrid", {'axes.grid' : False})
    g1 = sns.displot(data=df_all_level, 
                     col=c, 
                     x="Hyperscore", 
                     hue="decoy", 
                     binwidth=1.5, 
                     palette=['lightblue', 'red'], 
                     legend=False)
    #kde=True : curve insert above hist
    axes_subplots = g1.fig.axes
    for i, fdr_border in enumerate(border_list):
        subplot_axe = axes_subplots[i]
        subplot_axe.axvline(fdr_border, linewidth=2, color='black', label='FDR')
        y_min, y_max = subplot_axe.get_ylim()
        subplot_axe.text(fdr_border+1, y_max-(y_max/10), str(fdr_border), va='bottom')
    plt.legend(title='', loc='upper right', labels=[f'{fdr} FDR','DECOY', 'PSM'])
    # p.fig.suptitle
    plt.suptitle(title,y=1.05)
    plt.savefig(output, format='svg')

def create_histogramm(df_with_sep_decoy, output, fdr_border, fdr):
    """
    single histogramm
    """
    sns.set_style("whitegrid", {'axes.grid' : False})
    plt.figure(figsize=(20,6))
    sns.displot(df_with_sep_decoy, 
                x="Hyperscore", 
                hue="decoy", 
                binwidth=1.5, 
                palette=['lightblue', 'red'], 
                legend=False,)    
    plt.axvline(fdr_border, linewidth=2, color='black', label='FDR')
    axes = plt.gca()
    y_min, y_max = axes.get_ylim()
    plt.text(fdr_border+1, y_max-(y_max/10), str(fdr_border), va='bottom')
    plt.legend(title='', loc='upper right', labels=[f'{fdr} FDR','DECOY', 'PSM'])
    plt.suptitle('Reference Database')
    plt.savefig(output, format='svg')
    return plt
    #plt.close()

In [18]:
def create_specificity_and_sensitivity_side_by_side_plot(df_spe, df_sen, output):
    sns.set_theme(style="whitegrid")
    p=[sns.color_palette("colorblind")[1], sns.color_palette("colorblind")[0], sns.color_palette("colorblind")[2]]
    fig, axes = plt.subplots(1, 2, figsize=(14, 8))
    sns.lineplot(ax=axes[0], data=df_spe, x="level", y="specificity", hue='database', palette=p, legend=False)
    axes[0].set_title('Specificity')
    axes[0].set(ylim=(50, 100))
    sns.lineplot(ax=axes[1], data=df_sen, x="level", y="sensitivity", hue='database',  palette=p, legend=False)
    axes[1].set_title('Sensitivity')

    plt.legend(title='database', bbox_to_anchor=(1, 1), loc=2, labels=['NCBI-nr', 'UniProtKB-nr', 'Swiss-Prot'])
    plt.savefig(output, format='svg')

In [19]:
#or whitegrid?
#xlabels = ['axes.axisbelow':False]
def create_db_size_lineplot(DatabaseSize_df, file, output):
    uni_color=sns.color_palette("colorblind")[0]
    uni_nr_color=sns.color_palette()[9]
    ncbi_color=sns.color_palette("colorblind")[1]
    ncbi_rh_color = 'orange'
    swiss_color=sns.color_palette("colorblind")[2]
    palette1=[ncbi_color, uni_color, uni_nr_color, swiss_color]
    palette2=[ncbi_color, ncbi_rh_color,  uni_color, uni_nr_color, swiss_color]
    if 'kleiner' in file:
        title = '29MM'
    elif 'tanca' in file:
        title = '9MM'
    if 'noRH' in file:
        g = sns.catplot(x='level', y='# fasta', hue='database', linestyles=["-", "-", "-", "-"],
            kind="point", palette=palette1, data=DatabaseSize_df)  
        plt.ylim([0,(50000000)])
    else:    
        g = sns.catplot(x='level', y='size (GB)', hue='database', linestyles=["-", "-", "-","-", "-"],
            kind="point", palette=palette2, data=DatabaseSize_df)
        plt.ylim([0,43])
    g.fig.suptitle(title)
    g.savefig(f"{output}{Path(file +'.txt').stem}.svg", format='svg')

In [20]:
sns.set_style("white") 
def create_fasta_size_vs_identifed_psms_plot(df, output):
    plt.clf()
    if set(df['dataset']) == {'kleiner'}:
        title = '29MM'
        p=sns.color_palette("colorblind")[0:4]
        palette=[p[1], p[0], p[2], p[3]]
        #plt.ylim([0,30])
    elif set(df['dataset']) == {'Tanca'}:
        title = '9MM'
        p=sns.color_palette("colorblind")[0:3]
        palette=[p[1], p[0], p[2]]
        #plt.ylim([50,100])
    g = sns.scatterplot(x='# fasta (M)', y='% PSMs identified', style ='level', hue='database', palette=palette, linestyles=["-", "-"], data=df) 
    sns.lineplot(x='# fasta (M)', y='% PSMs identified', hue='database', legend=False,palette=palette, data=df) 
    g.set_title(title)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.tight_layout()
    plt.savefig(output, format='svg')

In [21]:
def create_database_level_vs_identified_barplot(df_for_plot, dataset, output, fdr=0.1):    
    title=''
    plt.clf()
    p=sns.color_palette("colorblind")[0:5]
    if fdr == 0.05:
        plt.ylim([0,50])
    elif fdr == 0.1:
        plt.ylim([0,60])
    if dataset == 'tanca':
        title= '9MM'
        plt.ylim([0,100])
        palette=[p[1], p[0], p[2]]
    elif dataset == 'kleiner':
        title= '29MM'
        palette=[p[1], p[0], p[2]]
    elif dataset =='reference':
        title= '29MM'      
        palette=[p[4]]
        
    
    
    #number = len(set(list(PSM['Database level'])))
    #g = sns.barplot(x="database level", y="% identified spectra", hue='database', palette=sns.color_palette("hls", 1), data=df_for_plot
    g = sns.barplot(x="database level", y="% identified spectra", hue='database', palette=palette, data=df_for_plot)
    handles, labels = g.get_legend_handles_labels()
    g.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    g.get_legend().set_visible(True)
    for tick in g.get_xticklabels():
        tick.set_rotation(0)       
    g.set_title(title)
    plt.tight_layout()
    plt.savefig(output, format='svg')

In [22]:
def create_identified_spectra_barplot(input_file_list):
    for i, file in enumerate(input_file_list):
        title=''
        plt.clf()   
        if 'tanca' in file:
            title= '9MM'
            plt.ylim([0,100])
        elif 'kleiner' in file:
            title= '29MM'
            plt.ylim([0,30])
        PSM_df = pd.read_csv(file+".txt", sep='\t')
        print(PSM_df)
        PSM_df["% PSM identified"]=get_percentage_identified_spectra_for_uniprot_ncbi_swissprot_all_level(0.05)
        number = len(set(list(PSM_df['Database level'])))
        g = sns.barplot(x="Database level", y="% PSM identified", hue='Database', palette=sns.color_palette("GnBu", 3), data=PSM)
        g.ylabel = '% identified spectra'
        handles, labels = g.get_legend_handles_labels()
        g.legend(loc='lower center', bbox_to_anchor=(1.2, 0.3))
        #g.get_legend().set_visible(False)
        for tick in g.get_xticklabels():
            tick.set_rotation(0)       
        g.set_title(title)
        plt.tight_layout()
        plt.savefig(input_file_list[i]+".svg", format='svg')

In [23]:
def create_nb_of_psms_per_species_barplot_and_csv(df, column_name, output, taxon_graph):
    new_column_name = f"{column_name.split('_')[1]}"
    df[new_column_name] = df[column_name].apply(lambda taxid: taxon_graph.get_scientific_name(taxid) if taxid not in ['CRAP', 'DECOY'] else taxid)
    plt = df.groupby([new_column_name]).size().plot(kind="bar", ylim=(0,18000), ylabel='# PSMs')
    
    plt.figure.savefig(output +".svg", format='svg', bbox_inches='tight')
    df.groupby([new_column_name]).size().to_csv(output +".csv", sep = '\t')

In [24]:
def create_psms_per_taxID_plot(path_to_reduced_df, path_to_bachelor_result, level, fdr, outputpath):
    # take old df and calculate '% PSM' new for all taxids
    sns.set_style("white")
    kleiner_palette_strain = [ 'red', 'lightcoral', 'grey', 'darkgrey', 'chocolate', 'mediumpurple','darkorchid','rebeccapurple',   'chocolate', 'limegreen', 'forestgreen', 'grey', 'sienna', 'chocolate', 'firebrick',  'dodgerblue', 'royalblue', 'gold', 'darkorange', 'orange', 'yellow', 'khaki', 'lemonchiffon', 'lightcoral', 'chocolate']
    kleiner_palette_species = [ 'red', 'lightcoral', 'grey', 'darkgrey', 'chocolate', 'mediumpurple','darkorchid','rebeccapurple', 'chocolate', 'limegreen', 'forestgreen', 'grey', 'sienna', 'firebrick',  'dodgerblue', 'royalblue', 'gold', 'darkorange', 'orange', 'yellow', 'lemonchiffon', 'lightcoral', 'chocolate']
    kleiner_palette_genus = [ 'red', 'lightcoral', 'grey', 'darkgrey', 'chocolate', 'rebeccapurple', 'chocolate', 'limegreen', 'forestgreen', 'grey', 'sienna', 'firebrick',  'dodgerblue', 'royalblue', 'gold', 'darkorange', 'orange', 'yellow', 'lemonchiffon', 'lightcoral', 'chocolate']
    kleiner_palette_family = [ 'red', 'lightcoral', 'grey','darkgrey', 'chocolate', 'rebeccapurple', 'chocolate',  'forestgreen', 'grey', 'firebrick',  'royalblue', 'gold', 'darkorange',  'lemonchiffon', 'lightcoral', 'chocolate']
    kleiner_palette_order = [ 'red', 'lightcoral', 'grey', 'darkgrey', 'chocolate', 'rebeccapurple', 'chocolate',  'forestgreen', 'grey', 'firebrick',  'royalblue', 'darkorange', 'lemonchiffon', 'lightcoral', 'chocolate']
    df = get_perc_per_taxa_df(path_to_reduced_df, path_to_bachelor_result, level, fdr)
    print(df)
    if level == 'subspecies':
        palette=kleiner_palette_strain
    elif level == 'species':
        palette=kleiner_palette_species
    elif level == 'genus':
        palette=kleiner_palette_genus
    elif level == 'family':
        palette=kleiner_palette_family
    elif level == 'order':
        palette=kleiner_palette_order
    if 'ncbi' in path_to_reduced_df:
        db = 'NCBI'
    elif 'uniprot' in path_to_reduced_df:
        db = 'Uniprot'
    elif 'swissprot' in path_to_reduced_df:
        db = 'Swissprot'
           
    sns.set_palette(sns.color_palette(palette))
    ax = df.set_index('taxon(s)').reindex(df.set_index('taxon(s)').sum().index, axis=1).T.plot(kind='bar', stacked=True, width=0.6,
          figsize=(18,9), fontsize=30)
    # ,title=("Species:Kleiner\ndatabase: %s   level: %s" %(db, level))
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(reversed(handles), reversed(labels), loc='lower center', bbox_to_anchor=(1.3, 0.1))
    ax.get_legend().set_visible(False)
    #ax.set_xticklabels("")
    for tick in ax.get_xticklabels():
        tick.set_rotation(0)
    plt.tight_layout()
    plt.ylim([0,100])
    fig = plt.gcf()
    fig.savefig(Path(outputpath)/(str(Path(path_to_reduced_df).stem)+f"_{fdr}_taxon_specific_identifications.svg"), format='svg')

In [25]:
def plot_lineplot_sens_spec(df_spe_sens, output):
    p=sns.color_palette("colorblind")[0:5]
    palette=[p[1], p[0], p[2]]
    plt.figure(figsize=(10,6))
    sns.set_theme(style="whitegrid")
    sns.lineplot(x='level', y='value', data=df_spe_sens, palette=palette, hue='database', style='sens/spe')
    plt.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0)
    plt.ylabel("specificity/sensitivity")
    plt.savefig(output,  bbox_inches='tight')
    return plt

In [26]:
def create_psm_identified_vs_db_size_with_ref(fdr, file):
    sns.set_style("white") 
    plt.clf()
    df = pd.read_csv(file, sep='\t')
    # df["% PSMs identified"] = df.apply(lambda row: get_percentage_identified_spectra(row.database, row.level, fdr), axis=1)
    # neu berechnet für fdr 0.05
    df["% PSMs identified"] = [35.384406, 44.396742, 40.370870, 39.526954, 36.972553, 43.847348, 40.353878]
    print(df)
    if 'kleiner' in file:
        title = '29MM'
        #plt.ylim([0,30])
    elif 'tanca' in file:
        title = '9MM'
        #plt.ylim([50,100])
    g = sns.scatterplot(x='# fasta (M)', y='% PSMs identified', style ='level', hue='database', palette=['red','blue'],legend=False, linestyles=["-", "-"], data=df) 
    sns.lineplot(x='# fasta (M)', y='% PSMs identified', hue='database', legend=False, palette=['red','blue'], data=df) 
    
    # Reference DB point
    ident_ref = get_percentage_identified_spectra('reference', '', 0.05)
    plt.scatter(123088/1000000, ident_ref, marker='o', color='black', label='reference DB', s=30)
    g.set_title(title)
    plt.tight_layout()
    plt.legend(title='', loc='upper right', labels=['UniProtKB', 'NCBI', 'reference DB'])
    plt.savefig(input[i]+".svg", format='svg')
    
    #g.fig.suptitle(title)
    #g.savefig(output[i]+".svg", format='svg')

In [27]:
def create_stacked_plot(df_from_analysis, output):
    taxa_arabidopsis = 3702
    df_from_analysis['mock community'] = df_from_analysis.apply(lambda x : True
            if x['taxID_species'] != taxa_arabidopsis and x['taxID_species'] != 'DECOY' else False, axis = 1)  
    # Count number of True in the series
   # num_ara= len(df_from_analysis['taxID_species'][df_from_analysis['taxID_species'] == {3702}].index)
    num_human = len(df_from_analysis['taxID_species'][df_from_analysis['taxID_species'] == {9606}].index)
    num_sacc = len(df_from_analysis['taxID_species'][df_from_analysis['taxID_species'] == {4932}].index)
    num_taxa = len(df_from_analysis['mock community'][df_from_analysis['mock community'] == True].index)
    num_decoy= len(df_from_analysis['decoy'][df_from_analysis['decoy'] == {True}].index)
    
    # Hyperscore values arabidopsis
   # df_from_analysis['ara'] = df_from_analysis.apply(lambda x : True
    #        if x['taxID_species'] == {3702} else False, axis = 1)  
    #score_ara = df_from_analysis['Hyperscore'][df_from_analysis['ara'] == True]
    #df_without_ara = df_from_analysis[['Hyperscore', 'decoy']][df_from_analysis['ara'] == False]
    #score_without_ara = df_from_analysis['Hyperscore'][df_from_analysis['decoy']=={False}]
    #print(score_ara)
    #print(score_without_ara.mean())
    # highest 47, lowest 21.1, mean 23.48, all mean = 40.11
    
    df = pd.DataFrame()
    df['PSMs'] = [num_taxa, num_decoy, num_human, num_sacc]
    print(df)
    
    #columns=["mock community", 'Decoy', 'Arabidopsis thaliana']
    ax = df.T.plot(kind='bar', stacked=True, ylabel='# PSM', title='29MM DB with junk sequences')
   # ax.legend(title='', loc='lower center', bbox_to_anchor=(1.3, 0.1), labels=['mock community', 'Decoy', 'Arabidopsis thaliana'])
    ax.legend(title='', loc='lower center', bbox_to_anchor=(1.3, 0.1), labels=['mock community', 'Decoy', 'Homo sapiens', 'Saccharomyces cerevisiae'])
    ax.figure.savefig(output, format='svg', bbox_inches='tight')

In [28]:
number_psm_result, df_from_analysis = get_psm_and_df_in_fdr(reference_tsv_with_aradiopsis, 0.05, remove_one_charged_spectra=True, columns=['taxID_species'])

In [29]:
number_psm_result, df_from_analysis_0_9 = get_psm_and_df_in_fdr(path_to_kleiner_species_nr_0_9_identity_reduced, 0.05, remove_one_charged_spectra=True, columns=['taxID_species'])

In [ ]:
create_stacked_plot(df_from_analysis, "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/arabiopsis_stacked_plot.svg")
print('number of sequences with arabiopsis: 139096, number of sequences without arabiopsis: 123204 , more seq: ', 100 - 123204/139096*100, '%')
print(f"{359/44176*100}% hits are arbiopsis PSMs, assumed false positives: {(359+44176)*0.05}")

In [ ]:
create_stacked_plot(df_more_junk_seq_from_analysis, "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/more_junk_stacked_plot.svg")
(519+182)/42596

In [ ]:
46620128 / 220000000 

In [ ]:
# create df sensitivity analysis and save
# create_all_analysis_result_df(0.05).to_csv('/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/sensitivity_analysis_0_05_fdr.tsv', sep = '\t')
create_all_analysis_result_df(0.1).to_csv('/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/sensitivity_analysis_0_1_fdr.tsv', sep = '\t')

In [ ]:
# load ref file
kleiner_pep_xml_result = pd.read_csv('/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/spectra/Run1_U1_2000ng.pep.xml.tsv', sep='\t')
kleiner_pep_xml_result = kleiner_pep_xml_result.rename(columns={'Peptide': 'Peptide_ref'})
kleiner_pep_xml_result_reduced_df = kleiner_pep_xml_result.groupby(["Title", "Ref_Score"], as_index=False).agg(
            {'ProteinAcc': lambda acc: set(acc), 'Peptide_ref': lambda seq: set(seq)})

In [ ]:
# create single histogramm for uniprot species db with 90% sequence identity
# 1. read df
identity_uni_species_df = ReferenceWriter.read_csv_with_generic_function(path_to_kleiner_species_nr_0_9_identity_reduced,['Protein', 'Hyperscore', 'decoy', 'taxID'])
# create df of right format
df_with_sep_decoy_ref = create_df_for_df_with_sep_decoy(identity_uni_species_df)
# get fdr_border
fdr = 0.05
result_file = f"/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/uniprot_species_sequences_0_9_identity/Run1_U1_2000ng_uniprot_species_0_9_identity.t.xml_new_reduced.tsv_0.05_sensitivity_new"
fdr_border = 23.9
# plot histogramm
create_histogramm(df_with_sep_decoy_ref, f'/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/histogramms/histogramm_psm_decoy_reference_{fdr}.svg', fdr_border, fdr)

In [ ]:
# create single histogramm for reference db
# 1. read df
kleiner_ref_db_df = ReferenceWriter.read_csv_with_generic_function("/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_kleiner_db/Run1_U1_2000ng.t.xml_reduced.tsv",['Protein', 'Hyperscore', 'decoy', 'taxID'])
# create df of right format
df_with_sep_decoy_ref = create_df_for_df_with_sep_decoy(kleiner_ref_db_df)
# get fdr_border
fdr = 0.1
result_file = f"/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/sensitivity_analysis_0_1_fdr.tsv"
fdr_border = read_value_from_result_file(result_file, 'reference', '-', 'lowest score in FDR')
# plot histogramm
create_histogramm(df_with_sep_decoy_ref, f'/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/histogramms/histogramm_psm_decoy_reference_{fdr}.svg', fdr_border, fdr)

In [ ]:
# histogram reference and referece aradiopsis
# create df for histogramm, get fdr_borders
levels = ['-']
dbs = ['reference', 'reference with aradiopsis seq', 'reference with ara yeast human seq']
reduced_tsv_dicts = [reference_tsv_with_kleiner_db, reference_tsv_with_aradiopsis, reference_tsv_with_ara_yeast_human]
df_for_hist = create_all_level_df_for_histogram_per_db(reduced_tsv_dicts, ['reference', 'reference + 11% false seq', 'reference + 200% false seq'])
# get fdr_border
for fdr in [0.1, 0.05]:
    result_file = f"/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/sensitivity_analysis_0_{str(fdr).split('.')[-1]}_fdr.tsv"
    border_list = []
    for db in dbs:
        border_list.append(read_value_from_result_file(result_file, db, levels[0], 'lowest score in FDR'))

    # plot and save histogramm
    output = f'/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/histogramms/histogramm_psm_decoy_reference_with_ara_{fdr}.svg'
    create_histograms_in_columns_per_db(df_for_hist, border_list, output, 'Reference', fdr, c="database")

In [ ]:
# histogramms with subspecies
levels = ['subspecies', 'species', 'genus', 'family']
db = 'uniprot_nr'
reduced_tsv_dict = uniprot_nr_reduced_tsv
df_for_hist = create_all_level_df_for_histogram_per_db(reduced_tsv_dict, levels)
# get fdr_border
for fdr in [0.1, 0.05]:
    result_file = f"/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/sensitivity_analysis_0_{str(fdr).split('.')[-1]}_fdr.tsv"
    border_list = []
    for level in levels:
        border_list.append(read_value_from_result_file(result_file, db, level, 'lowest score in FDR'))
    # plot and save histogramm
    output = f'/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/histogramms/histogramm_psm_decoy_{db}_with_subspecies_{fdr}.svg'
    create_histograms_in_columns_per_db(df_for_hist, border_list, output, 'UniProtKB', fdr)


In [ ]:
# create df for histogramm, get fdr_borders
levels = ['subspecies', 'species', 'genus']
db = 'ncbi'
reduced_tsv_dict = ncbi_reduced_tsv
df_for_hist = create_all_level_df_for_histogram_per_db(reduced_tsv_dict, levels)
# get fdr_border
for fdr in [0.1, 0.05]:
    result_file = f"/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/sensitivity_analysis_0_{str(fdr).split('.')[-1]}_fdr.tsv"
    border_list = []
    for level in levels:
        border_list.append(read_value_from_result_file(result_file, db, level, 'lowest score in FDR'))

    # plot and save histogramm
    output = f'/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/histogramms/histogramm_psm_decoy_{db}_with_subspecies_{fdr}.svg'
    create_histograms_in_columns_per_db(df_for_hist, border_list, output, 'NCBI', fdr)



In [ ]:
# create df for histogramm, get fdr_borders
levels = ['species', 'genus', 'family', 'order']
db = 'swissprot'
reduced_tsv_dict = swissprot_reduced_tsv
df_for_hist = create_all_level_df_for_histogram_per_db(reduced_tsv_dict, levels)
# get fdr_border
for fdr in [0.1, 0.05]:
    result_file = f"/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/sensitivity_analysis_0_{str(fdr).split('.')[-1]}_fdr.tsv"
    border_list = []
    for level in levels:
        border_list.append(read_value_from_result_file(result_file, db, level, 'lowest score in FDR'))

    # plot and save histogramm
    output = f'/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/histogramms/histogramm_psm_decoy_{db}_{fdr}.svg'
    create_histograms_in_columns_per_db(df_for_hist, border_list, output, 'SwissProt', fdr)


In [ ]:
output = f'/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/histogramms/histogramm_psm_decoy_ncbi_with_subspecies.svg'
create_histograms_in_columns_per_db(df_ncbi, border_list_ncbi, output, 'NCBI')

In [ ]:
output = f'/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/histogramms/histogramm_psm_decoy_swissprot.svg'
create_histograms_in_columns_per_db(df_swissprot, border_list_swissprot, output, 'SwissProt')

In [ ]:
# ROC
fdrs = [0.001, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
files = [f'/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/uniprot/x_tandem_tsv/Run1_U1_2000ng_uniprot_species_nr.t.xml_reduced.tsv_{fdr}_sensitivity'
        for fdr in fdrs]

files_ignore_unclassified = [f'/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/uniprot/x_tandem_tsv/Run1_U1_2000ng_uniprot_species_nr.t.xml_reduced.tsv_ignore_unclassified_{fdr}_sensitivity' 
                            for fdr in fdrs]
used_files=files_ignore_unclassified
TP_values = [read_TP(file) for file in used_files]
FN_values = [read_FN(file) for file in used_files]
FP_values = [read_FP(file) for file in used_files]
TN_values = [read_TN(file) for file in used_files]
#fpr = fp/(fp+tn)
fpr_values = [FP_values[i]/(FP_values[i]+TN_values[i]) for i in range(len(used_files))]
#tpr = tp/(tp+fn) = sensitivity
tpr_values =  [TP_values[i]/(TP_values[i]+FN_values[i]) for i in range(len(used_files))]
m = max(tpr_values)
maxes = [i for i, j in enumerate(tpr_values) if j == m]
print('tpr max:', [fdrs[i] for i in maxes])
#print(tpr_values)
fig, ax = plt.subplots(figsize=(10,7))
ax.plot(fpr_values, tpr_values)
#ax.plot(np.linspace(0, 1, 100),
 #        np.linspace(0, 1, 100),
  #       label='baseline',
   #      linestyle='--')
plt.title('Receiver Operating Characteristic Curve', fontsize=18)
plt.ylabel('TPR', fontsize=16)
plt.xlabel('FPR', fontsize=16)
#plt.legend(fontsize=12);

In [ ]:
# sensitivity and specificity plot
fdr=0.1
dict_databases_to_size_and_result_file = get_dict_databases_to_size_and_result_file(fdr)
column_names = ['database', 'level', 'specificity', 'sensitivity']
rows =[['UniProtKB-nr', 'strain', read_specificity(dict_databases_to_size_and_result_file['uniprot_subspecies'][1]), 
        read_sensitivity(dict_databases_to_size_and_result_file['uniprot_subspecies'][1])], 
    ['UniProtKB-nr', 'species', read_specificity(dict_databases_to_size_and_result_file['uniprot_species_nr'][1]), 
        read_sensitivity(dict_databases_to_size_and_result_file['uniprot_species_nr'][1])], 
       ['UniProtKB-nr', 'genus', read_specificity(dict_databases_to_size_and_result_file['uniprot_genus_nr'][1]), 
        read_sensitivity(dict_databases_to_size_and_result_file['uniprot_genus_nr'][1])], 
       ['UniProtKB-nr', 'family', read_specificity(dict_databases_to_size_and_result_file['uniprot_family_nr'][1]), 
        read_sensitivity(dict_databases_to_size_and_result_file['uniprot_family_nr'][1])],
       ['NCBI-nr', 'strain', read_specificity(dict_databases_to_size_and_result_file['ncbi_subspecies'][1]), 
        read_sensitivity(dict_databases_to_size_and_result_file['ncbi_subspecies'][1])],
      ['NCBI-nr', 'species', read_specificity(dict_databases_to_size_and_result_file['ncbi_species'][1]), 
        read_sensitivity(dict_databases_to_size_and_result_file['ncbi_species'][1])],
       ['NCBI-nr', 'genus', read_specificity(dict_databases_to_size_and_result_file['ncbi_genus'][1]), 
        read_sensitivity(dict_databases_to_size_and_result_file['ncbi_genus'][1])],
      ['Swiss-Prot', 'species', read_specificity(dict_databases_to_size_and_result_file['swissprot_species'][1]), 
        read_sensitivity(dict_databases_to_size_and_result_file['swissprot_species'][1]) ], 
       ['Swiss-Prot', 'genus', read_specificity(dict_databases_to_size_and_result_file['swissprot_genus'][1]), 
        read_sensitivity(dict_databases_to_size_and_result_file['swissprot_genus'][1]) ], 
       ['Swiss-Prot', 'family', read_specificity(dict_databases_to_size_and_result_file['swissprot_family'][1]), 
        read_sensitivity(dict_databases_to_size_and_result_file['swissprot_family'][1]) ]]
df_spe_sens = pd.DataFrame(rows,columns=column_names)
df_spe = df_spe_sens[['database', 'level', 'specificity']]
df_sen = df_spe_sens[['database', 'level', 'sensitivity']]
df_spe.database=pd.Categorical(df_spe.database,categories=['NCBI-nr', 'UniProtKB-nr', 'Swiss-Prot'])
df_spe=df_spe.sort_values('database')
df_sen.database=pd.Categorical(df_sen.database,categories=['NCBI-nr', 'UniProtKB-nr', 'Swiss-Prot'])
df_sen=df_sen.sort_values('database')
create_specificity_and_sensitivity_side_by_side_plot(df_spe, df_sen, "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/sens_spec_0_05.svg")


In [ ]:
# specificity and sensitivity with unidentified spectra
def create_df_for_lineplot_from_rows_df2(column_names, rows):
    df_spe_sens = pd.DataFrame(rows,columns=column_names)
    return df_spe_sens

fdr = 0.05
dict_databases_to_size_and_result_file = get_dict_databases_to_size_and_result_file(fdr)
column_names = ['database', 'level', 'value', 'sens/spe']
rows =[
    ['NCBI-nr', 'strain', read_specificity(dict_databases_to_size_and_result_file['ncbi_subspecies'][1]), 'specificity'], 
    ['NCBI-nr', 'strain', read_sensitivity(dict_databases_to_size_and_result_file['ncbi_subspecies'][1]), 'sensitivity'],
    ['NCBI-nr', 'species', read_specificity(dict_databases_to_size_and_result_file['ncbi_species'][1]), 'specificity'], 
    ['NCBI-nr', 'species', read_sensitivity(dict_databases_to_size_and_result_file['ncbi_species'][1]), 'sensitivity'],
    ['NCBI-nr', 'genus', read_specificity(dict_databases_to_size_and_result_file['ncbi_genus'][1]), 'specificity'], 
    ['NCBI-nr', 'genus',read_sensitivity(dict_databases_to_size_and_result_file['ncbi_genus'][1]), 'sensitivity'],
    ['UniProtKB-nr', 'strain', read_specificity(dict_databases_to_size_and_result_file['uniprot_subspecies'][1]), 'specificity'],
    ['UniProtKB-nr', 'strain', read_sensitivity(dict_databases_to_size_and_result_file['uniprot_subspecies'][1]), 'sensitivity'], 
    ['UniProtKB-nr', 'species', read_specificity(dict_databases_to_size_and_result_file['uniprot_species_nr'][1]), 'specificity'], 
    ['UniProtKB-nr', 'species', read_sensitivity(dict_databases_to_size_and_result_file['uniprot_species_nr'][1]), 'sensitivity'], 
    ['UniProtKB-nr', 'genus', read_specificity(dict_databases_to_size_and_result_file['uniprot_genus_nr'][1]), 'specificity'], 
    ['UniProtKB-nr', 'genus',read_sensitivity(dict_databases_to_size_and_result_file['uniprot_genus_nr'][1]), 'sensitivity'], 
    ['UniProtKB-nr', 'family', read_specificity(dict_databases_to_size_and_result_file['uniprot_family_nr'][1]), 'specificity'], 
    ['UniProtKB-nr', 'family', read_sensitivity(dict_databases_to_size_and_result_file['uniprot_family_nr'][1]), 'sensitivity'],
    ['Swiss-Prot', 'species', read_specificity(dict_databases_to_size_and_result_file['swissprot_species'][1]), 'specificity'], 
    ['Swiss-Prot', 'species', read_sensitivity(dict_databases_to_size_and_result_file['swissprot_species'][1]) , 'sensitivity'],
    ['Swiss-Prot', 'genus', read_specificity(dict_databases_to_size_and_result_file['swissprot_genus'][1]), 'specificity'], 
    ['Swiss-Prot', 'genus', read_sensitivity(dict_databases_to_size_and_result_file['swissprot_genus'][1]), 'sensitivity' ], 
    ['Swiss-Prot', 'family', read_specificity(dict_databases_to_size_and_result_file['swissprot_family'][1]), 'specificity'], 
    ['Swiss-Prot', 'family',read_sensitivity(dict_databases_to_size_and_result_file['swissprot_family'][1]), 'sensitivity' ]
]
df_spe_sens = pd.DataFrame(rows,columns=column_names)

plt = plot_lineplot_sens_spec(df_spe_sens, '/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/specificity_sensitivity_0_05.png')

In [ ]:
# database size
db_size_files = ["/home/jules/Documents/Bachelorarbeit/Graphiken/db_size/database_size_kleiner_noRH",
       "/home/jules/Documents/Bachelorarbeit/Graphiken/db_size/database_size_kleiner",
       "/home/jules/Documents/Bachelorarbeit/Graphiken/db_size/database_size_tanca",
       "/home/jules/Documents/Bachelorarbeit/Graphiken/db_size/database_size_tanca_noRH"]
sns.set_style("white") 
output = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/database_size/"
for i, file in enumerate(db_size_files):
        DatabaseSize_df = pd.read_csv(file+".txt", sep='\t')
        DatabaseSize_df['database'] = DatabaseSize_df['database'].replace(['UniProt_NR'],'UniProtKB-nr')
        DatabaseSize_df['database'] = DatabaseSize_df['database'].replace(['UniProt'],'UniProtKB')
        DatabaseSize_df['database'] = DatabaseSize_df['database'].replace(['NCBI-nr'],'NCBI-nr')
        create_db_size_lineplot(DatabaseSize_df, file, output)



In [ ]:
number_tanca_spectra = 8471
tanca_nb_identified_spectra_0_05 = {}
for (db_dict, db) in zip([tanca_swissprot_tsv, tanca_uniprot_tsv, tanca_ncbi_tsv], ['swissprot', 'uniprot', 'ncbi']):
    tanca_nb_identified_spectra_0_05[db] = {}
    for k, v in db_dict.items():
        number_psm_result, df = get_psm_and_df_in_fdr(v, 0.05, remove_one_charged_spectra=True, columns=[f'taxid_{k}'])
        df_without_decoy = df[get_hit_rows2(df.decoy)]
        number_spectra = len(set(df_without_decoy.Title))
        tanca_nb_identified_spectra_0_05[db][k] = (number_spectra, number_spectra/number_tanca_spectra*100)
print(tanca_nb_identified_spectra_0_05)


In [ ]:
#tanca_nb_identified_spectra_0_1 = tanca_nb_identified_spectra
df_for_plot_tanca = pd.read_csv("/home/jules/Documents/Bachelorarbeit/Graphiken/db_size/db_size_psm_tanca.txt", sep="\t")
df_for_plot_tanca.database=pd.Categorical(df_for_plot_tanca.database,categories=['NCBI', 'UniProtKB_NR', 'SwissProt'])
df_for_plot_tanca=df_for_plot_tanca.sort_values('database')
psm_identified = [tanca_nb_identified_spectra_0_05['ncbi']['species'][1],tanca_nb_identified_spectra_0_05['ncbi']['genus'][1],
                 tanca_nb_identified_spectra_0_05['ncbi']['family'][1],
                  tanca_nb_identified_spectra_0_05['uniprot']['species'][1],tanca_nb_identified_spectra_0_05['uniprot']['genus'][1],
                 tanca_nb_identified_spectra_0_05['uniprot']['family'][1], tanca_nb_identified_spectra_0_05['uniprot']['order'][1],
                 tanca_nb_identified_spectra_0_05['swissprot']['species'][1],tanca_nb_identified_spectra_0_05['swissprot']['genus'][1],
                 tanca_nb_identified_spectra_0_05['swissprot']['family'][1], tanca_nb_identified_spectra_0_05['swissprot']['order'][1],]

df_for_plot_tanca['% PSMs identified'] = psm_identified
print(df_for_plot_tanca)

In [ ]:
# database-size vs. PSMs
full_df = create_all_analysis_result_df(0.1)
number_of_spectra = 164414
number_of_spectra_passing_quality_control = 88279
PSM_identified_uniprot = get_psm_per_db_from_sens_analysis(full_df, 'uniprot')
PSM_identified_ncbi = get_psm_per_db_from_sens_analysis(full_df, 'ncbi')
PSM_identified_swissprot = get_psm_per_db_from_sens_analysis(full_df, 'swissprot')
PSM_identified_ncbi_percentage = [psm/number_of_spectra_passing_quality_control * 100 for psm in PSM_identified_ncbi]
PSM_identified_uniprot_percentage= [psm/number_of_spectra_passing_quality_control * 100 for psm in PSM_identified_uniprot]
PSM_identified_swissprot_percentage= [psm/number_of_spectra_passing_quality_control * 100 for psm in PSM_identified_swissprot]
uniprot_database_sizes = [uniprot_nr_db[0]/1000000 for uniprot_nr_db in uniprot_nr_dict.values()]
ncbi_database_sizes = [ncbi_db[0]/1000000 for ncbi_db in ncbi_dict.values()]
swissprot_database_sizes = [swissprot_db[0]/1000000 for swissprot_db in swissprot_dict.values()]


In [ ]:
# database-size vs. PSMs Plots
# identified PSMs deutlich abweichend von Bachelor-Analyse
# columns:  # fasta (M)    level  % PSMs identified   database  dataset    
df_for_plot_kleiner = create_pd_df_for_psm_plot(PSM_identified_uniprot_percentage, PSM_identified_ncbi_percentage, 
                                                uniprot_database_sizes, ncbi_database_sizes, PSM_identified_swissprot_percentage, 
                                                swissprot_database_sizes)
all_result_df = create_all_analysis_result_df(0.1)
#print(all_result_df)
df_reference = pd.DataFrame([[0.1, 'species', 51320/number_of_spectra_passing_quality_control*100, 'Reference', "kleiner"]], columns=["# fasta (M)", "level", "% PSMs identified", "database", "dataset"])
df_for_plot_kleiner = df_for_plot_kleiner.append(df_reference)
# kleiner
df_for_plot_kleiner['database'] = df_for_plot_kleiner['database'].replace(['UniProtKB'],'UniProtKB-nr')
df_for_plot_kleiner['database'] = df_for_plot_kleiner['database'].replace(['Swissprot'],'Swiss-Prot')
df_for_plot_kleiner['database'] = df_for_plot_kleiner['database'].replace(['NCBI'],'NCBI-nr')
df_for_plot_kleiner.database=pd.Categorical(df_for_plot_kleiner.database,categories=['NCBI-nr', 'UniProtKB-nr', 'Swiss-Prot', 'Reference'])
df_for_plot_kleiner=df_for_plot_kleiner.sort_values('database')
create_fasta_size_vs_identifed_psms_plot(df_for_plot_kleiner, 
                                         "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/database_size/kleiner_uni_ncbi_db_psm_vs_db_size_0_1_with_reference.svg")

#tanca
df_for_plot_tanca['database'] = df_for_plot_tanca['database'].replace(['SwissProt'],'Swiss-Prot')
df_for_plot_tanca['database'] = df_for_plot_tanca['database'].replace(['UniProtKB_NR'],'UniProtKB-nr')
df_for_plot_tanca['database'] = df_for_plot_tanca['database'].replace(['NCBI'],'NCBI-nr')
# create_fasta_size_vs_identifed_psms_plot(df_for_plot_tanca, "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/database_size/tanca_uni_ncbi_db_psm_vs_db_size_0_05.svg")

In [ ]:
#PSMs 
outputfile_kl = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/PSM/kleiner_psm_vs_level_0_05.svg"
# created psm vs dbsize plot
#  # fasta (M) database level  % identified spectra   database  dataset
df_for_plot_all_db = create_pd_df_for_psm_plot(PSM_identified_uniprot_percentage, PSM_identified_ncbi_percentage, uniprot_database_sizes, ncbi_database_sizes, PSM_identified_swissprot_percentage, swissprot_database_sizes)
df_for_plot_all_db = df_for_plot_all_db[df_for_plot_all_db.level != 'order']
df_for_plot_all_db = df_for_plot_all_db.rename(columns={'level': 'database level', '% PSMs identified': "% identified spectra"})
df_for_plot_all_db.database=pd.Categorical(df_for_plot_all_db.database,categories=['NCBI', 'UniProtKB', 'Swissprot'])
df_for_plot_all_db=df_for_plot_all_db.sort_values('database')
df_for_plot_all_db['database'] = df_for_plot_all_db['database'].replace(['UniProtKB'],'UniProtKB-nr')
df_for_plot_all_db['database'] = df_for_plot_all_db['database'].replace(['Swissprot'],'Swiss-Prot')
df_for_plot_all_db['database'] = df_for_plot_all_db['database'].replace(['NCBI'],'NCBI-nr')
print(df_for_plot_all_db)
#create_database_level_vs_identified_barplot(df_for_plot_all_db, 'kleiner', outputfile_kl, 0.05)



In [ ]:
df_reference_0_1 = pd.DataFrame([[0.1, 'species', 51320/number_of_spectra_passing_quality_control*100, 'Reference', "kleiner"]], columns=["# fasta (M)", "database level", "% PSMs identified", "database", "dataset"])
df_reference_0_05 = pd.DataFrame([[0.1, 'species', 43482/number_of_spectra_passing_quality_control*100, 'Reference', "kleiner"]], columns=["# fasta (M)", "database level", "% PSMs identified", "database", "dataset"])
df_reference_0_05 = df_reference_0_05.rename(columns={'level': 'database level', '% PSMs identified': "% identified spectra"})
print(df_reference_0_05)
create_database_level_vs_identified_barplot(df_reference_0_05, 'reference', "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/PSM/reference_psm_0_05.svg", 0.05)

In [ ]:
df_for_plot_tanca = pd.read_csv("/home/jules/Documents/Bachelorarbeit/Graphiken/db_size/db_size_psm_tanca.txt", sep="\t")
df_for_plot_tanca['% PSMs identified'] = psm_identified
outputfile_tc = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/PSM/tanca_psm_vs_level_0_05.svg"
df_for_plot_all_db_tanca = df_for_plot_tanca.rename(columns={'level': 'database level', '% PSMs identified': "% identified spectra"})
df_for_plot_all_db_tanca.database=pd.Categorical(df_for_plot_all_db_tanca.database,categories=['NCBI', 'UniProtKB_NR', 'SwissProt'])
df_for_plot_all_db_tanca=df_for_plot_all_db_tanca.sort_values('database')
df_for_plot_all_db_tanca['database'] = df_for_plot_all_db_tanca['database'].replace(['UniProtKB_NR'],'UniProtKB-nr')
df_for_plot_all_db_tanca['database'] = df_for_plot_all_db_tanca['database'].replace(['SwissProt'],'Swiss-Prot')
df_for_plot_all_db_tanca['database'] = df_for_plot_all_db_tanca['database'].replace(['NCBI'],'NCBI-nr')
print(df_for_plot_all_db_tanca)
create_database_level_vs_identified_barplot(df_for_plot_all_db_tanca, 'tanca', outputfile_tc)

In [ ]:
set1 = set(['A', 'B', 'C', 'D'])
set2 = set(['B', 'C', 'D', 'E'])

v=venn2([set1, set2], ('Set1', 'Set2'))
v.get_patch_by_id('A').set_alpha(1.0)
#v.get_patch_by_id('100').set_color('white')
v.get_label_by_id('A').set_text('Unknown')
v.get_label_by_id('B').set_text('Set "A"')
plt.title("Sample Venn diagram")
plt.annotate('Unknown set', xy=v.get_label_by_id('A').get_position() - np.array([0, 0.05]), xytext=(-70,-70),
             ha='center', textcoords='offset points', bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.1),
             arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.5',color='gray'))

plt.show()

In [ ]:
# load spectra set for venn diagramm uniprot
fdr = 0.05
spectra_in_fdr_uniprot_species =  set(get_df_in_fdr_without_decoy(uniprot_nr_reduced_tsv['species'], fdr)['Title']) 
spectra_in_fdr_uniprot_species_species = set(get_df_in_fdr_without_decoy(uniprot_species_reduced_tsv['species'], fdr)['Title'])  
spectra_in_fdr_uniprot_genus =  set(get_df_in_fdr_without_decoy(uniprot_nr_reduced_tsv['genus'], fdr)['Title'])
spectra_in_fdr_uniprot_genus_species =  set(get_df_in_fdr_without_decoy(uniprot_species_reduced_tsv['genus'], fdr)['Title'])

In [ ]:
# load spectra set for venn diagramm ncbi
fdr = 0.05
spectra_in_fdr_ncbi_species =  set(get_df_in_fdr_without_decoy(ncbi_reduced_tsv['species'], fdr)['Title'])
spectra_in_fdr_ncbi_species_species =  set(get_df_in_fdr_without_decoy(ncbi_species_reduced_tsv['species'], fdr)['Title'])
spectra_in_fdr_ncbi_genus =  set(get_df_in_fdr_without_decoy(ncbi_reduced_tsv['genus'], fdr)['Title'])
spectra_in_fdr_ncbi_genus_species =  set(get_df_in_fdr_without_decoy(ncbi_species_reduced_tsv['genus'], fdr)['Title'])

In [ ]:
def get_venn_plot(setA, setB, position):
    total = len(setA.union(setB))
    v = venn2([setA, setB], set_labels = ('', ''), set_colors=['red', 'yellow'],
              subset_label_formatter=lambda x: f"{(x/total):1.2%}", ax=axes[position[0]][position[1]])
    v.get_patch_by_id('A').set_alpha(1.0)
    v.get_patch_by_id('B').set_alpha(1.0)
    return v

figure, axes = plt.subplots(2, 2)
v = get_venn_plot(spectra_in_fdr_uniprot_species, spectra_in_fdr_uniprot_species_species, (0,0))
v = get_venn_plot(spectra_in_fdr_uniprot_genus, spectra_in_fdr_uniprot_genus_species, (0,1))
v = get_venn_plot(spectra_in_fdr_ncbi_species, spectra_in_fdr_ncbi_species_species, (1,0))
v = get_venn_plot(spectra_in_fdr_ncbi_genus, spectra_in_fdr_ncbi_genus_species, (1,1))

figure.legend(title='', loc='upper center', labels=['all descandants', 'no sequences below species level'])
plt.savefig("/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/venn/uniprot_ncbi_venn2.svg", format='svg')
plt.show()

In [ ]:
total=1000
s = lambda x: f"{(x/total):1.2%}"
s(999)

In [ ]:
# 92470
merged_df_u, final_decoy_list_u, identified_spectra_of_all_levels_together_u = get_merged_df_of_all_levels(uniprot_nr_reduced_tsv, 0.1, ["subspecies", "species", "genus", "family"])
perc_all_u_0_05 = identified_spectra_of_all_levels_together_u/number_of_spectra_passing_quality_control*100
print(perc_all_u_0_05)
path = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/uniprot_all_level_merged.tsv"
# merged_df_u[final_decoy_list_u][['Title', 'decoy_subspecies', 'decoy_species', 'decoy_genus', 'decoy_family']].to_csv(path, sep='\t')

In [ ]:
merged_df_n, final_decoy_list_n, identified_spectra_of_all_levels_together_n = get_merged_df_of_all_levels(ncbi_reduced_tsv, 0.1, ["subspecies", "species", "genus"])
perc_all_n_0_1 = identified_spectra_of_all_levels_together_n/number_of_spectra_passing_quality_control*100
print(perc_all_n_0_1)

In [ ]:
merged_df_s, final_decoy_list_s, identified_spectra_of_all_levels_together_s = get_merged_df_of_all_levels(swissprot_reduced_tsv, 0.1, ["species", "genus", "family", "order"])
perc_all_s_0_1 = identified_spectra_of_all_levels_together_s/number_of_spectra_passing_quality_control*100
print(perc_all_s_0_1)

In [ ]:
def add_united_values(df, uni_united, ncbi_united, swiss_united=None):
    # columns =['# fasta (M)', 'level', '% PSMs identified', 'database', 'dataset']
    df.loc[df.index.max() + 1] = ['', 'united', uni_united, 'UniProtKB-nr', 'kleiner']
    df.loc[df.index.max() + 1] = ['', 'united', ncbi_united, 'NCBI-nr', 'kleiner']
    df.loc[df.index.max() + 1] = ['', 'united', swiss_united, 'Swiss-Prot', 'kleiner']
    return df
perc_all_u_0_05 = 50.17274776560677
perc_all_n_0_05 = 49.22914849511209
perc_all_s_0_05 = 29.03861620543957
perc_all_u_0_1 = 58.515615265238615
perc_all_n_0_1 = 57.611663022915984
perc_all_s_0_1 = 33.93219225410347
df_for_plot_all_db_with_united = add_united_values(df_for_plot_all_db, perc_all_u_0_1, perc_all_n_0_1, perc_all_s_0_1)
#  # fasta (M) database level  % identified spectra   database  dataset
df_for_plot_all_db_with_united.columns = ['# fasta (M)', 'database level', '% identified spectra', 'database', 'dataset']

print(df_for_plot_all_db_with_united)
create_database_level_vs_identified_barplot(df_for_plot_all_db_with_united, 'kleiner', "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/PSM/kleiner_psm_vs_level_united_0_1.svg")

In [ ]:
# deviation PSM new calculated
path_to_mean_deviation_df = ["/home/jules/Documents/Metaproteomics/Bachelorarbeit/Graphiken/taxon_specific_identification&db/mean_deviation_kleiner",
        "/home/jules/Documents/Metaproteomics/Bachelorarbeit/Graphiken/taxon_specific_identification&db/mean_deviation_tanca"]
sns.set_style("white") 

for file in path_to_mean_deviation_df:
    #deviation_df = create_mean_deviation_df(file)
    deviation_df = pd.read_csv(file)
    print(deviation_df)
    if 'kleiner' in file:
        title = '29MM'
        g = sns.catplot(x='level', y='mean deviation', hue='database', linestyles=["-", "-", "-"], kind="point", palette=sns.color_palette("colorblind")[0:4], data=deviation_df)  
       # plt.ylim([0,5])
    elif 'tanca' in file:
        title = '9MM'
        g = sns.catplot(x='level', y='mean deviation', hue='database', linestyles=["--", "--", "--"], kind="point", palette=sns.color_palette("colorblind")[0:4], data=deviation_df)  
    plt.ylim([0,13])
    plt.ylabel('mean deviation (%)')
    #g.fig.suptitle(title)
    g.savefig(file+".svg", format='svg')

In [ ]:
# deviation PSM bases on bachelor results
path_to_mean_deviation_df = ["/home/jules/Documents/Bachelorarbeit/Graphiken/taxon_specific_identification&db/mean_deviation_kleiner",
        "/home/jules/Documents/Bachelorarbeit/Graphiken/taxon_specific_identification&db/mean_deviation_tanca"]
sns.set_style("white") 

for file in path_to_mean_deviation_df:
    deviation_df = pd.read_csv(file+".txt", sep='\t')
    
    deviation_df['database'] = deviation_df['database'].replace(['UniProt'],'UniProtKB-nr')
    deviation_df.database=pd.Categorical(deviation_df.database,categories=['NCBI-nr', 'UniProtKB-nr', 'Swiss-Prot'])
    deviation_df=deviation_df.sort_values('database')
    p = [sns.color_palette('colorblind')[1], sns.color_palette('colorblind')[0], sns.color_palette('colorblind')[2]]
    print(deviation_df)
    if 'kleiner' in file:
        title = '29MM'
        g = sns.catplot(x='level', y='mean deviation', hue='database', linestyles=["-", "-", "-"], kind="point", palette=p, data=deviation_df)  
        save_file= "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/mean_deviation/mean_deviation_kleiner.svg"
    # plt.ylim([0,5])
    elif 'tanca' in file:
        title = '9MM'
        g = sns.catplot(x='level', y='mean deviation', hue='database', linestyles=["--", "--", "--"], kind="point", palette=p, data=deviation_df)  
        save_file= "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/mean_deviation/mean_deviation_tanca.svg"
    plt.ylim([0,13])
    plt.ylabel('mean deviation (%)')
    #g.fig.suptitle(title)
    g.savefig(save_file, format='svg')

In [ ]:
input_files = {
    'uniprot':
    [(uniprot_nr_reduced_tsv["subspecies"], 
      "/home/jules/Documents/Metaproteomics/Bachelorarbeit/Graphiken/taxon_specific_identification&db/Kleiner_unidb_strain.txt", "subspecies"),
     (uniprot_nr_reduced_tsv["species"],
      "/home/jules/Documents/Metaproteomics/Bachelorarbeit/Graphiken/taxon_specific_identification&db/Kleiner_unidb_species.txt", "species"),
    (uniprot_nr_reduced_tsv["genus"],
     "/home/jules/Documents/Metaproteomics/Bachelorarbeit/Graphiken/taxon_specific_identification&db/Kleiner_unidb_genus.txt","genus"),
    ( uniprot_nr_reduced_tsv["family"],
     "/home/jules/Documents/Metaproteomics/Bachelorarbeit/Graphiken/taxon_specific_identification&db/Kleiner_unidb_family.txt", "family")],
    "ncbi":
    [(ncbi_reduced_tsv["subspecies"],
     "/home/jules/Documents/Metaproteomics/Bachelorarbeit/Graphiken/taxon_specific_identification&db/Kleiner_ncbidb_strain.txt", "subspecies"),
     (ncbi_reduced_tsv["species"],
      "/home/jules/Documents/Metaproteomics/Bachelorarbeit/Graphiken/taxon_specific_identification&db/Kleiner_ncbidb_species.txt", "species"),
     (ncbi_reduced_tsv["genus"],
      "/home/jules/Documents/Metaproteomics/Bachelorarbeit/Graphiken/taxon_specific_identification&db/Kleiner_ncbidb_genus.txt", "genus")],
     "swissprot":
    [(swissprot_reduced_tsv["species"],
      "/home/jules/Documents/Metaproteomics/Bachelorarbeit/Graphiken/taxon_specific_identification&db/Kleiner_swissprot_species.txt", 'species'),
    (swissprot_reduced_tsv["genus"],
     "/home/jules/Documents/Metaproteomics/Bachelorarbeit/Graphiken/taxon_specific_identification&db/Kleiner_swissprot_genus.txt", "genus"),
    (swissprot_reduced_tsv["family"],
     "/home/jules/Documents/Metaproteomics/Bachelorarbeit/Graphiken/taxon_specific_identification&db/Kleiner_swissprot_family.txt", "family"),
    (swissprot_reduced_tsv["order"],
     "/home/jules/Documents/Metaproteomics/Bachelorarbeit/Graphiken/taxon_specific_identification&db/Kleiner_swissprot_order.txt", "order")]
}

other_files = ["/home/jules/Documents/Metaproteomics/Bachelorarbeit/Graphiken/taxon_specific_identification&db/Kleiner_ncbidb_genus_species",
              ]
outputpath = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/taxon_specific_identification/"
fdr = 0.1
for files in input_files['swissprot']:
    create_psms_per_taxID_plot(files[0], files[1], files[2], fdr, outputpath)
fdr = 0.1
for files in input_files['ncbi']:
    create_psms_per_taxID_plot(files[0], files[1], files[2], fdr, outputpath)
fdr = 0.1
for files in input_files['uniprot']:
    create_psms_per_taxID_plot(files[0], files[1], files[2], fdr, outputpath)


In [ ]:
# % PSM identified barplot
#PSMs 
input_files = [
    "/home/jules/Documents/Metaproteomics/Bachelorarbeit/Graphiken/PSM/PSM_kleiner_ncbi_vs_uniprot_vs_swissprot",
    # "/home/jules/Documents/Metaproteomics/Bachelorarbeit/Graphiken/PSM/PSM_tanca_ncbi_vs_uniprot_vs_swissprot"
]
create_identified_spectra_barplot(input_files)


In [ ]:
# database-size vs. PSMs
input_files = ["/home/jules/Documents/Metaproteomics/Bachelorarbeit/Graphiken/db_size/db_size_psm_tanca.txt",
        "/home/jules/Documents/Metaproteomics/Bachelorarbeit/Graphiken/db_size/db_size_psm_kleiner.txt"]
create_psm_identified_vs_db_size_with_ref(0.05, input_files[1])

In [ ]:
from number_of_psm_per_species import PsmNumberPerTaxIDs
obj = PsmNumberPerTaxIDs('kleiner', input_files['uniprot'][0][0], input_files['uniprot'][0][2])
taxID_to_spectra_dict = obj.count_row_by_row()
taxID_to_spectra_dict = obj.get_virus_spectra(taxID_to_spectra_dict)
taxID_to_percentage_dict = obj.get_percentage(taxID_to_spectra_dict)
print(taxID_to_percentage_dict)

In [ ]:
uni_spec_in_fdr = get_df_in_fdr(uniprot_reduced_tsv_species_df, 0.1)
# kleiner_pep_xml_result:  Title ProteinAcc  Peptide  Ref_Score
# uni_spec_in_fdr: reduced_df of uniprot species matches in FDR-border: Title Peptide  Hyperscore taxID  decoy 
print('number of identified spectra by Kleiner: ', len(set(kleiner_pep_xml_result['Title'].tolist())))
print('number of identified spectra by Uniprot species: ', len(set(uni_spec_in_fdr['Title'].tolist())))
# Title  Ref_Score  ProteinAcc Peptide_x (set)  Peptide_y(string)  Hyperscore   taxID    decoy
merged_pep_xml_uniprot_df = pd.merge(kleiner_pep_xml_result_reduced_df, uni_spec_in_fdr, how="outer", left_on='Title', right_on="Title")
df_identified_in_uniprot = merged_pep_xml_uniprot_df[merged_pep_xml_uniprot_df['Peptide_ref'].isna()]
df_identified_in_kleiner_ref = merged_pep_xml_uniprot_df[merged_pep_xml_uniprot_df['Peptide'].isna()]
print('number of spectra identified in uniprot species result but not in kleiner: ', len(set(df_identified_in_uniprot['Title'].tolist())))
print('number of spectra identified in kleiner but not in uniprot species result: ', len(set(df_identified_in_kleiner_ref['Title'].tolist())))

In [ ]:
# test of df with one charge spectra and so

# now all reduced dataframes without charge one spectra (_new_reduced)
uniprot_reduced_tsv_species_nr_new_df = ReferenceWriter.read_csv_with_generic_function(uniprot_nr_reduced_tsv['species'],['Protein', 'Hyperscore', 'decoy', 'taxID'], remove_one_charged_spectra=True)
print('number spectra of new reduced df: ',len(uniprot_reduced_tsv_species_nr_new_df))
uniprot_reduced_tsv_species_nr_new_df = ReferenceWriter.read_csv_with_generic_function(uniprot_nr_reduced_tsv['species'],['Protein', 'Hyperscore', 'decoy', 'taxID'], remove_one_charged_spectra=False)
print('number spectra of new reduced df (removed one charge spectra): ',len(uniprot_reduced_tsv_species_nr_new_df))
uniprot_reduced_tsv_species_nr_old_df = ReferenceWriter.read_csv_with_generic_function( "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/uniprot/x_tandem_tsv/Run1_U1_2000ng_uniprot_species_nr.t.xml_reduced.tsv",['Protein', 'Hyperscore', 'decoy', 'taxID'], remove_one_charged_spectra=False)
print('number spectra of old reduced df (with one charge spectra): ',len(uniprot_reduced_tsv_species_nr_old_df))
uniprot_reduced_tsv_species_nr_old_df = ReferenceWriter.read_csv_with_generic_function( "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/uniprot/x_tandem_tsv/Run1_U1_2000ng_uniprot_species_nr.t.xml_reduced.tsv",
                                                                                       ['Protein', 'Hyperscore', 'decoy', 'taxID'], remove_one_charged_spectra=True)
print('number spectra of old reduced df (removed one charge spectra): ', len(uniprot_reduced_tsv_species_nr_old_df))
# check if same df, also new analysis old analysis
# a = pd.merge(uniprot_reduced_tsv_species_nr_new_df[['Title', 'Peptide']], uniprot_reduced_tsv_species_nr_old_df[['Title', 'Peptide']], how="outer", on='Title')
# Test for differences old and new analysis
excluded_spectra = set(a[a.Peptide_x.isna()]['Title']) # 46701
print(len(set(a[a.Peptide_y.isna()]['Title']))) # 0
excluded_spectra_with_number_at_last_pos = set([spectrum for spectrum in a[a.Peptide_x.isna()]['Title'] if spectrum.split('.')[-1] != '' ])
included_spectra = uniprot_reduced_tsv_species_nr_new_df['Title']
included_spectra_without_number_at_last_pos = set([spectrum for spectrum in included_spectra if spectrum.split('.')[-1] == '' ])
print(included_spectra_without_number_at_last_pos)
excluded_spectra_without_number_at_last_pos = set([spectrum for spectrum in excluded_spectra if spectrum.split('.')[-1] == '' ])
print(len(excluded_spectra))
print(a[a.Peptide_y == 'VGDYTVAIGNPFGLGETVTSGIVSALGR']) # Spectrum Run1_U1_2000ng.151046.151046. und Run1_U1_2000ng.151255.151255. not passed quality control new analysis
uniprot_reduced_tsv_genus_nr_new_df = ReferenceWriter.read_csv_with_generic_function(uniprot_nr_reduced_tsv['genus'],['Protein', 'Hyperscore', 'decoy', 'taxID'])
intersect = set(uniprot_reduced_tsv_genus_nr_new_df['Title']).intersection(excluded_spectra)
print(sorted(list(excluded_spectra_with_number_at_last_pos)))
print(sorted(list(intersect)))
ncbi_reduced_tsv_subspecies_old_df = ReferenceWriter.read_csv_with_generic_function( ncbi_reduced_tsv['subspecies'], ['Protein', 'Hyperscore', 'decoy', 'taxID'])
print(len(ncbi_reduced_tsv_subspecies_old_df))
ncbi_reduced_tsv_subspecies_new = remove_spectra_of_charge_one_from_reduced_tsv(ncbi_reduced_tsv_subspecies_old_df)
print(len(ncbi_reduced_tsv_subspecies_new))

In [ ]:
uni_spec_in_fdr = uni_spec_in_fdr.rename(columns={'Peptide': 'Peptide_uni'})
merged_pep_xml_uniprot_df = pd.merge(kleiner_pep_xml_result_reduced_df, uni_spec_in_fdr, how="outer", on="Title")
a = pd.merge(merged_pep_xml_uniprot_df_er, merged_pep_xml_uniprot_df, how="outer", on='Title')
b = a[a['Peptide_uni'].isna()][['Title', 'Peptide_ref_x', 'Peptide_ref_y', 'Peptide_uni', 'Peptide_er']]
c = b[b['Peptide_er'].notna()]
d = c[c['Peptide_ref_x'].notna()]
# without quality control: 8 specta more identified (equal to kleiner_ref), all other trash

In [1]:
# load taxon graph
import sys  
sys.path.insert(0, '/home/jules/tax2proteome_projects/tax2proteome/')
from TaxonGraph import TaxonGraph
taxon_graph = TaxonGraph()
taxon_graph.create_graph("/home/jules/Documents/Metaproteomics/databases/databases_tax2proteome/taxdump.tar.gz")

In [ ]:
taxon_graph.find_taxIDs()

In [ ]:
file=uniprot_nr_reduced_tsv['species']
fdr=0.1
df_uni_species = get_psm_and_df_in_fdr(file, fdr, remove_one_charged_spectra=True, columns=["taxID_species"])[1]
df_uni_species_without_set = df_uni_species.explode('taxID_species').reset_index(drop=True)
df_uni_species_without_crap = df_uni_species_without_set[[True if t_set != "CRAP" else False for t_set in df_uni_species_without_set.taxID_species]]
df_uni_species_without_decoy = df_uni_species_without_crap[[True if t_set != "DECOY" else False for t_set in df_uni_species_without_crap.taxID_species]]

create_nb_of_psms_per_species_barplot_and_csv(df_uni_species_without_decoy, 'taxID_species', '/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/nb_psm_per_species_barplot_and_csv/nb_psm_per_species_uni_0_1', taxon_graph)

In [ ]:
file=uniprot_nr_reduced_tsv['genus']
fdr=0.1
df_uni_genus = get_psm_and_df_in_fdr(file, fdr, remove_one_charged_spectra=True, columns=["taxID_genus"])[1]
df_uni_genus_without_set = df_uni_genus.explode('taxID_genus').reset_index(drop=True)
df_uni_genus_without_crap = df_uni_genus_without_set[[True if t_set != "CRAP" else False for t_set in df_uni_genus_without_set.taxID_genus]]
df_uni_genus_without_decoy = df_uni_genus_without_crap[[True if t_set != "DECOY" else False for t_set in df_uni_genus_without_crap.taxID_genus]]

create_nb_of_psms_per_species_barplot_and_csv(df_uni_genus_without_decoy, 'taxID_genus', '/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/nb_psm_per_species_barplot_and_csv/nb_psm_per_genus_uni_0_1', taxon_graph)

In [ ]:
file=uniprot_nr_reduced_tsv['family']
fdr=0.1
df_uni_family = get_psm_and_df_in_fdr(file, fdr, remove_one_charged_spectra=True, columns=["taxID_family"])[1]
df_uni_family_without_set = df_uni_family.explode('taxID_family').reset_index(drop=True)
df_uni_family_without_crap = df_uni_family_without_set[[True if t_set != "CRAP" else False for t_set in df_uni_family_without_set.taxID_family]]
df_uni_family_without_decoy = df_uni_family_without_crap[[True if t_set != "DECOY" else False for t_set in df_uni_family_without_crap.taxID_family]]

create_nb_of_psms_per_species_barplot_and_csv(df_uni_family_without_decoy, 'taxID_family', '/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/nb_psm_per_species_barplot_and_csv/nb_psm_per_family_uni_0_1', taxon_graph)

In [ ]:
print(len(df_uni_family_without_set))
len(df_uni_family_without_decoy)
create_nb_of_psms_per_species_barplot_and_csv(df_uni_family_without_decoy, 'taxID_family', '/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/plots/nb_psm_per_species_barplot_and_csv/nb_psm_per_family_uni_0_05', taxon_graph)

In [14]:
# Vergleich bacillus.. species vs. family
# bacillus species taxon ID: 1423, family taxon ID: 186817
# staphylococcus: 1280, 90964
# Paracoccus denitrificans 266, 31989
# species_df mean score: 41.4, median: 38.5; family_df mean score: 44.1, median: 41.1
fdr=0.05
df_in_fdr_uniprot_species = get_df_in_fdr_without_decoy(uniprot_nr_reduced_tsv['species'], fdr, columns=['taxID_species']) 
df_in_fdr_uniprot_family =  get_df_in_fdr_without_decoy(uniprot_nr_reduced_tsv['family'], fdr, columns=['taxID_family'])
print(df_in_fdr_uniprot_family.mean())
print(df_in_fdr_uniprot_family.median())

In [ ]:
df_in_fdr_uniprot_species.head()
df_species_1423 = df_in_fdr_uniprot_species[get_taxa_rows(df_in_fdr_uniprot_species.taxID_species, 1423)]
df_family_186817 = df_in_fdr_uniprot_family[get_taxa_rows(df_in_fdr_uniprot_family.taxID_family, 186817)]

In [ ]:
spectra_set_1423 = set(df_species_1423['Title'])
spectra_set_186817 = set(df_family_186817['Title'])
print(str(len(spectra_set_1423)) + ' ' + str(len(spectra_set_186817)))
# waren alle spectra in family schon vorher identifiziert? von (2352-855) neu hinzugekommen, 221 komplett neu
spectra_set_new_in_186817=spectra_set_186817.difference(set(df_in_fdr_uniprot_species['Title']))

In [33]:
def count_taxa(df_column):
    taxa_count_dict = {}
    for tax_set in df_column:
        for taxon in tax_set:
            if taxon in taxa_count_dict:
                taxa_count_dict[taxon]+= 1
            else:
                taxa_count_dict[taxon]=1
    return taxa_count_dict

def get_taxon_specific_accs(tax, db):
    taxon_accs = set()
    len_db=len(db)
    all_descending_taxa = taxon_graph.find_taxIDs(tax)
    search_strs = [f'OX={taxon}' for taxon in all_descending_taxa]
    print(len(all_descending_taxa))
    for i, entry in enumerate(db):
        if (len_db/(i+1))%10==0:
            print(f"{(i+1)/len_db *100} % db searched")
        if any(taxon in entry.description for taxon in search_strs):
            taxon_accs.add(entry.id)
    return taxon_accs

def get_taxon_specific_accs2(tax, df_acc_taxon):
    taxon_accs = set()
    all_descending_taxa = taxon_graph.find_taxIDs(tax)
    print(len(all_descending_taxa))
    for taxon in all_descending_taxa:
        taxon_accs.add(entry.id)
    return taxon_accs
    

In [ ]:
ref_db = list(SeqIO.parse("/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/Kleiner_ref_db/Kleiner_ref_crap.fasta", "fasta"))
uni_s_db = list(SeqIO.parse("/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/Tax2Proteome_DB/kleiner_species_nr_crap.fasta", "fasta"))


In [32]:
def get_accs(tax):
    all_descending_taxa = taxon_graph.find_taxIDs(tax)
    acc_set = set()
    with open("/home/jules/Documents/Metaproteomics/databases/databases_tax2proteome/acc2tax_uniprot", 'r') as infile:
        for line in infile:
            fields = line.split('\t')
            try: 
                taxon = int(fields[1].strip())
                if taxon in all_descending_taxa:
                    acc_set.add(fields[0].strip())      
            except:
                continue
    return acc_set


In [30]:
def compare_seq(acc1, acc2):
    seq1 = wget(f"https://www.uniprot.org/uniref/{acc1}.fasta")
    seq2 = wget(f"https://www.uniprot.org/uniref/{acc2}.fasta")
    global_align = pw2.align.globalxx(first_seq, second_seq)
    number_matching_AS = global_align[0][2]
    percent_matcing = number_matching_seq/len(seq1)*100
    return percent_matching

87041


In [ ]:
# welche spectra neu dazu für bacillus auf family level
df_family_186817_not_in_species_df = df_family_186817[get_not_spectra_rows(df_family_186817.Title, spectra_set_1423)]
df_family_186817_not_in_species_df.head()
new_identified_peptides = set(df_family_186817_not_in_species_df.Peptide)

df_family_186817_not_in_species_df = df_family_186817[~df_family_186817.Title.isin(spectra_set_1423)]
df_only_one_taxa = df_family_186817_not_in_species_df[df_family_186817_not_in_species_df.taxID_family == {186817}]
print(len(df_only_one_taxa.Title))
#print((df_family_186817_not_in_species_df.taxID_family))
taxa_count_dict_186815 = count_taxa(df_family_186817_not_in_species_df.taxID_family)
print(taxa_count_dict_186815)    
new_identified_accs = [acc for acc_set in list(df_family_186817_not_in_species_df.Protein) for acc in acc_set]
# print(*list(set([acc.split('|')[1] for acc in flat_list])), sep = " OR ") 

#welche spectra schon vorher bekannt #2370, mean=42.6, median=39.8, 172576 different accs
df_family_186817_in_species_df = df_family_186817[get_not_spectra_rows(df_family_186817.Title, spectra_set_new_in_186817)]
print(len(df_family_186817_not_in_species_df))
flat_set = set([acc for acc_set in list(df_family_186817_in_species_df.Protein) for acc in acc_set])
print(len(new_identified_accs)) 
print(len(new_identified_peptides))

In [17]:
# bacillus subtilis spec:1423 fam: 186817
# welche spectra komplett neu, Score Werte von 101.1 - 28.3, mean=33.8, median=30.5, davon 8 ribosomale Proteine,
# 100 enzyme, 29 uncharacterized, 3 Flagellar (von 285), ATP: 48
# anzahl unterschiedlicher Peptide von 1816 spectra, 846 unterschiediche Peptide und 466753 neue accs
# davon 450 speziell nur für bacillus, 904 auch in 543, 617 in 125621, 453 un 90964
print("bacillus subtilis ")
df_species_1423 = df_in_fdr_uniprot_species[get_taxa_rows(df_in_fdr_uniprot_species.taxID_species, 1423)]
df_family_186817 = df_in_fdr_uniprot_family[get_taxa_rows(df_in_fdr_uniprot_family.taxID_family, 186817)]
print(f"identified spectra species: {len(set(df_species_1423.Title))} identified spectra family: {len(set(df_family_186817.Title))}")
df_family_186817_not_in_species_df=df_family_186817[~df_family_186817.Title.isin(df_species_1423.Title)]
print(f"new identified spectra family: {len(set(df_family_186817_not_in_species_df.Title))}")
df_only_one_taxa_186817 = df_family_186817_not_in_species_df[df_family_186817_not_in_species_df.taxID_family == {186817}]
taxa_count_dict_186817 = count_taxa(df_family_186817_not_in_species_df.taxID_family)
print(f"identified only for taxon: {len(set(df_only_one_taxa_186817.Title))}\n all spectra counts per taxa: {taxa_count_dict_186817}")
df_family_186817_in_species_df = df_family_186817_not_in_species_df[df_family_186817_not_in_species_df.Title.isin(df_in_fdr_uniprot_species.Title)]
print(f"davon wurde bereits vorher für andere spezien identifiziert: {len(set(df_family_186817_in_species_df.Title))} spektren.")

bacillus subtilis 
identified spectra species: 855 identified spectra family: 2352
new identified spectra family: 1712
identified only for taxon: 429
 all spectra counts per taxa: {186817: 1816, 72275: 405, 90964: 452, 543: 904, 32033: 545, 31989: 434, 135621: 617, 119060: 440, 1499392: 232, 206379: 199, 82115: 189, 3051: 11, 188786: 21, 194924: 48, 10662: 4, 10699: 6, 1033997: 1, 'DECOY': 6}
davon wurde bereits vorher für andere spezien identifiziert: 1491 spektren.


In [62]:
# wie oft neu identifizierte in allen spektren
def get_spectra_to_accs(df, accs, name, df_species):
    spectra_set =set()
    for row in df.values.tolist():
        if any(acc in row[1] for acc in accs):
            spectra_set.add(row[0])
    print(f'In species df identified PSM with {name}: {len(df_species[df_species.Title.isin(spectra_set)])}')

def get_new_spectra_characteristics(fam_taxon, df_family_taxon_not_in_species_df, df_family, df_species):
    taxa_spec_accs = get_accs(186817)
    new_identified_accs = [acc.split('|')[1] for acc_set in list(df_family_taxon_not_in_species_df.Protein) for acc in acc_set]
    #new_identified_accs = [acc.split('|')[1] for acc in new_identified_accs]
    new_identified_accs_186817 = taxa_spec_accs.intersection(set(new_identified_accs))
    print(f"Number of new identified accs: {len(new_identified_accs_186817)}")
    
    acc_to_count_dict = {}
    for acc in new_identified_accs_186817:
        count = sum([acc in acc_long for acc_long_set in df_family_taxon_not_in_species_df.Protein for acc_long in acc_long_set])
        acc_to_count_dict[acc]=count
    
    acc_count_2_dict={}
    acc_count_1_dict={}
    acc_count_3_dict={}
    acc_count_4_dict={}
    acc_count_5_dict={}
    for acc, count in acc_to_count_dict.items():
        if count ==1 :
            acc_count_1_dict[acc]=count
        elif count==2:
            acc_count_2_dict[acc]=count
        elif count==3:
            acc_count_3_dict[acc]=count
        elif count==4:
            acc_count_4_dict[acc]=count
        elif count>4:
            acc_count_5_dict[acc]=count
    print(f"mit 1 identifikation: {len(acc_count_1_dict)}, mit 2 identifikationen: {len(acc_count_2_dict)},\n\
    mit 3 identifikationen: {len(acc_count_3_dict)}, mit 4 identifikationen: {len(acc_count_4_dict)}, \n\
    mit mehr als 4 identifikationen: {len(acc_count_5_dict)}") 
    # davon schon im species df identifiziere spektren
    df = df_family[['Title','Protein']]
    df = df.astype({'Protein': 'string'})


    get_spectra_to_accs(df, acc_count_1_dict.keys(), 'count_1', df_species)
    get_spectra_to_accs(df, acc_count_2_dict.keys(), 'count_2', df_species)
    get_spectra_to_accs(df, acc_count_3_dict.keys(), 'count_3', df_species)
    get_spectra_to_accs(df, acc_count_4_dict.keys(), 'count_4', df_species)
    get_spectra_to_accs(df, acc_count_5_dict.keys(), 'count_5', df_species)


In [63]:
get_new_spectra_characteristics(186817, df_family_186817_not_in_species_df, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species)

Number of new identified accs: 4459
mit 1 identifikation: 2793, mit 2 identifikationen: 543,
    mit 3 identifikationen: 496, mit 4 identifikationen: 97, 
    mit mehr als 4 identifikationen: 530
In species df identified PSM with count_1: 412
In species df identified PSM with count_2: 322
In species df identified PSM with count_3: 168
In species df identified PSM with count_4: 105
In species df identified PSM with count_5: 1381


In [46]:
# chromobacterium violaceum spec: 536 fam: 1499392
print("chromobacterium violaceum ")
df_species_536 = df_in_fdr_uniprot_species[get_taxa_rows(df_in_fdr_uniprot_species.taxID_species, 536)]
df_family_1499392 = df_in_fdr_uniprot_family[get_taxa_rows(df_in_fdr_uniprot_family.taxID_family, 1499392)]
print(f"identified spectra species: {len(set(df_species_536.Title))} identified spectra family: {len(set(df_family_1499392.Title))}")
df_family_1499392_not_in_species_df=df_family_1499392[~df_family_1499392.Title.isin(df_species_536.Title)]
print(f"new identified spectra family: {len(set(df_family_1499392_not_in_species_df.Title))}")
df_only_one_taxa_1499392 = df_family_1499392_not_in_species_df[df_family_1499392_not_in_species_df.taxID_family == {1499392}]
taxa_count_dict_1499392 = count_taxa(df_family_1499392_not_in_species_df.taxID_family)
print(f"identified only for taxon: {len(set(df_only_one_taxa_1499392.Title))}\n all spectra counts per taxa: {taxa_count_dict_1499392}")
df_family_1499392_in_species_df = df_family_1499392_not_in_species_df[df_family_1499392_not_in_species_df.Title.isin(df_in_fdr_uniprot_species.Title)]
print(f"davon wurde bereits vorher für andere spezien identifiziert: {len(set(df_family_1499392_in_species_df.Title))} spektren")

chromobacterium violaceum 
identified spectra species: 1129 identified spectra family: 1534
new identified spectra family: 677
identified only for taxon: 95
 all spectra counts per taxa: {1499392: 715, 32033: 271, 186817: 165, 135621: 372, 72275: 193, 119060: 459, 31989: 186, 543: 246, 206379: 165, 90964: 83, 82115: 99, 194924: 46, 188786: 48, 3051: 4, 10699: 1, 'DECOY': 4, 10744: 1, 1033997: 1}
davon wurde bereits vorher für andere spezien identifiziert: 649 spektren


In [65]:
get_new_spectra_characteristics(1499392, df_family_1499392_not_in_species_df, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species)

Number of new identified accs: 1708
mit 1 identifikation: 531, mit 2 identifikationen: 465,
    mit 3 identifikationen: 405, mit 4 identifikationen: 278, 
    mit mehr als 4 identifikationen: 29
In species df identified PSM with count_1: 41
In species df identified PSM with count_2: 70
In species df identified PSM with count_3: 50
In species df identified PSM with count_4: 23
In species df identified PSM with count_5: 680


In [67]:
# paracoccus dentrificans: spec: 266 fam: 31989
print("paracoccus dentrificans")
df_species_266 = df_in_fdr_uniprot_species[get_taxa_rows(df_in_fdr_uniprot_species.taxID_species, 266)]
df_family_31989 = df_in_fdr_uniprot_family[get_taxa_rows(df_in_fdr_uniprot_family.taxID_family, 31989)]
print(f"identified spectra species: {len(set(df_species_266.Title))} identified spectra family: {len(set(df_family_31989.Title))}")
df_family_31989_not_in_species_df=df_family_31989[~df_family_31989.Title.isin(df_species_266.Title)]
print(f"new identified spectra family: {len(set(df_family_31989_not_in_species_df.Title))}")
df_only_one_taxa_31989 = df_family_31989_not_in_species_df[df_family_31989_not_in_species_df.taxID_family == {31989}]
taxa_count_dict_31989 = count_taxa(df_family_31989_not_in_species_df.taxID_family)
print(f"identified only for taxon: {len(set(df_only_one_taxa_31989.Title))}\n all spectra counts per taxa: {taxa_count_dict_31989}")
df_family_31989_in_species_df = df_family_31989_not_in_species_df[df_family_31989_not_in_species_df.Title.isin(df_in_fdr_uniprot_species.Title)]
print(f"davon wurde bereits vorher für andere spezien identifiziert: {len(set(df_family_31989_in_species_df.Title))} spektren")

paracoccus dentrificans
identified spectra species: 614 identified spectra family: 3026
new identified spectra family: 2583
identified only for taxon: 975
 all spectra counts per taxa: {82115: 734, 31989: 2895, 1499392: 469, 186817: 451, 135621: 1042, 90964: 387, 119060: 746, 543: 776, 32033: 603, 72275: 523, 194924: 141, 206379: 255, 10744: 4, 3051: 26, 10662: 6, 188786: 19, 10699: 10, 1033997: 3, 'DECOY': 11}
davon wurde bereits vorher für andere spezien identifiziert: 1963 spektren


In [68]:
get_new_spectra_characteristics(31989, df_family_31989_not_in_species_df, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species)

Number of new identified accs: 11958
mit 1 identifikation: 7454, mit 2 identifikationen: 1968,
    mit 3 identifikationen: 824, mit 4 identifikationen: 438, 
    mit mehr als 4 identifikationen: 1274
In species df identified PSM with count_1: 477
In species df identified PSM with count_2: 186
In species df identified PSM with count_3: 143
In species df identified PSM with count_4: 114
In species df identified PSM with count_5: 898


In [69]:
# staphylococcus aureus: spec: 1280 fam: 90964
print("staphylococcus aureus")
df_species_1280 = df_in_fdr_uniprot_species[get_taxa_rows(df_in_fdr_uniprot_species.taxID_species, 1280)]
df_family_90964 = df_in_fdr_uniprot_family[get_taxa_rows(df_in_fdr_uniprot_family.taxID_family, 90964)]
print(f"identified spectra species: {len(set(df_species_1280.Title))} identified spectra family: {len(set(df_family_90964.Title))}")
df_family_90964_not_in_species_df=df_family_90964[~df_family_90964.Title.isin(df_species_1280.Title)]
print(f"new identified spectra family: {len(set(df_family_90964_not_in_species_df.Title))}")
df_only_one_taxa_90964 = df_family_90964_not_in_species_df[df_family_90964_not_in_species_df.taxID_family == {90964}]
taxa_count_dict_90964 = count_taxa(df_family_90964_not_in_species_df.taxID_family)
print(f"identified only for taxon: {len(set(df_only_one_taxa_90964.Title))}\n all spectra counts per taxa: {taxa_count_dict_90964}")
df_family_90964_in_species_df = df_family_90964_not_in_species_df[df_family_90964_not_in_species_df.Title.isin(df_in_fdr_uniprot_species.Title)]
print(f"davon wurde bereits vorher für andere spezien identifiziert: {len(set(df_family_90964_in_species_df.Title))} spektren")

staphylococcus aureus
identified spectra species: 928 identified spectra family: 2850
new identified spectra family: 2203
identified only for taxon: 46
 all spectra counts per taxa: {90964: 2217, 543: 2114, 186817: 560, 72275: 423, 32033: 314, 82115: 117, 31989: 182, 135621: 465, 1499392: 198, 194924: 26, 119060: 161, 10744: 13, 3051: 16, 206379: 89, 10662: 9, 10699: 20, 188786: 34, 'DECOY': 3}
davon wurde bereits vorher für andere spezien identifiziert: 2162 spektren


In [70]:
get_new_spectra_characteristics(90964, df_family_90964_not_in_species_df, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species)

Number of new identified accs: 2564
mit 1 identifikation: 1338, mit 2 identifikationen: 689,
    mit 3 identifikationen: 274, mit 4 identifikationen: 7, 
    mit mehr als 4 identifikationen: 256
In species df identified PSM with count_1: 72
In species df identified PSM with count_2: 62
In species df identified PSM with count_3: 94
In species df identified PSM with count_4: 68
In species df identified PSM with count_5: 837


In [71]:
# altermonas macleodii spec: 28108, fam: 72275
print("altermonas macleodii")
df_species_28108 = df_in_fdr_uniprot_species[get_taxa_rows(df_in_fdr_uniprot_species.taxID_species, 28108)]
df_family_72275 = df_in_fdr_uniprot_family[get_taxa_rows(df_in_fdr_uniprot_family.taxID_family, 72275)]
print(f"identified spectra species: {len(set(df_species_28108.Title))} identified spectra family: {len(set(df_family_72275.Title))}")
df_family_72275_not_in_species_df=df_family_72275[~df_family_72275.Title.isin(df_species_28108.Title)]
print(f"new identified spectra family: {len(set(df_family_72275_not_in_species_df.Title))}")
df_only_one_taxa_72275 = df_family_72275_not_in_species_df[df_family_72275_not_in_species_df.taxID_family == {72275}]
taxa_count_dict_72275 = count_taxa(df_family_72275_not_in_species_df.taxID_family)
print(f"identified only for taxon: {len(set(df_only_one_taxa_72275.Title))}\n all spectra counts per taxa: {taxa_count_dict_72275}")
df_family_72275_in_species_df = df_family_72275_not_in_species_df[df_family_72275_not_in_species_df.Title.isin(df_in_fdr_uniprot_species.Title)]
print(f"davon wurde bereits vorher für andere spezien identifiziert: {len(set(df_family_72275_in_species_df.Title))} spektren")

altermonas macleodii
identified spectra species: 895 identified spectra family: 2224
new identified spectra family: 1562
identified only for taxon: 293
 all spectra counts per taxa: {186817: 279, 72275: 1785, 90964: 361, 543: 1008, 135621: 785, 1499392: 213, 32033: 545, 206379: 188, 119060: 501, 31989: 363, 82115: 175, 3051: 25, 188786: 44, 10744: 11, 194924: 82, 10699: 6, 10662: 4, 'DECOY': 5, 1033997: 1}
davon wurde bereits vorher für andere spezien identifiziert: 1432 spektren


In [72]:
get_new_spectra_characteristics(72275, df_family_72275_not_in_species_df, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species)

Number of new identified accs: 5149
mit 1 identifikation: 2137, mit 2 identifikationen: 320,
    mit 3 identifikationen: 1153, mit 4 identifikationen: 82, 
    mit mehr als 4 identifikationen: 1457
In species df identified PSM with count_1: 133
In species df identified PSM with count_2: 58
In species df identified PSM with count_3: 112
In species df identified PSM with count_4: 20
In species df identified PSM with count_5: 833


In [ ]:
# loading 9MM species and family:
fdr=0.05
tanca_species_df = get_df_in_fdr_without_decoy("/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/9MM_FASP/x_tandem_tsv/9MM_FASP_uniprot_Tanca_species_nr.t.xml_new_reduced.tsv", fdr, columns=['taxID_species'])
tanca_genus_df = get_df_in_fdr_without_decoy("/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/9MM_FASP/x_tandem_tsv/9MM_FASP_uniprot_Tanca_family_nr.t.xml_new_reduced.tsv", fdr, columns=['taxID_family'])

In [ ]:
# rhodotorula glutinis spec: 5535, genus: 5533
print("rhodotorula glutinis")
df_species_5535 = tanca_species_df[get_taxa_rows(tanca_species_df.taxID_species, 5535)]
df_genus_5533 = tanca_genus_df[get_taxa_rows(tanca_genus_df.taxID_genus, 5533)]
print(f"identified spectra species: {len(set(df_species_5535.Title))} identified spectra genu: {len(set(df_genus_5533.Title))}")
df_genus_5533_not_in_species_df=df_genus_5533[~df_genus_5533.Title.isin(df_species_5535.Title)]
print(f"new identified spectra family: {len(set(df_family_72275_not_in_species_df.Title))}")
df_only_one_taxa_72275 = df_family_72275_not_in_species_df[df_family_72275_not_in_species_df.taxID_family == {72275}]
taxa_count_dict_72275 = count_taxa(df_family_72275_not_in_species_df.taxID_family)
print(f"identified only for taxon: {len(set(df_only_one_taxa_72275.Title))}\n all spectra counts per taxa: {taxa_count_dict_72275}")
df_family_72275_in_species_df = df_family_72275_not_in_species_df[df_family_72275_not_in_species_df.Title.isin(df_in_fdr_uniprot_species.Title)]
print(f"davon wurde bereits vorher für andere spezien identifiziert: {len(set(df_family_72275_in_species_df.Title))} spektren")

In [ ]:
len(set(df_family_186817_not_in_species_df.Peptide))

In [ ]:
df_in_fdr_uniprot_species[df_in_fdr_uniprot_species.Title == 'Run1_U1_2000ng.153630.153630.4']

In [ ]:
# compare additional identified reference to uniprot species
fdr=0.05
df_in_fdr_ref = get_df_in_fdr_without_decoy(reference_tsv_with_kleiner_db, fdr, columns=['taxID_species']) 

In [ ]:
print(len(set(df_in_fdr_ref.Title)))
print(len(set(df_in_fdr_ref[df_in_fdr_ref.decoy != {True}].Title)))


In [ ]:
df_in_fdr_uniprot_species = get_df_in_fdr_without_decoy(uniprot_nr_reduced_tsv['species'], fdr, columns=['taxID_species']) 
cs = ['Protein', 'Hyperscore', 'decoy', 'taxID', 'taxID_species']
df_full_uniprot_species = ReferenceWriter.read_csv_with_generic_function(uniprot_nr_reduced_tsv['species'], cs, remove_one_charged_spectra=True)

In [ ]:
print(len(set(df_in_fdr_uniprot_species.Title)))
print(len(set(df_in_fdr_uniprot_species[df_in_fdr_uniprot_species.decoy != {True}].Title)))

In [ ]:
ref_db = list(SeqIO.parse("/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/Kleiner_ref_db/Kleiner_ref_crap.fasta", "fasta"))
    

In [ ]:
def add_accs_with_no_diamond_result(ref_db, set_to_compare):
    missing_accs = set()
    for entry in ref_db:
        if entry.id not in set_to_compare:
            missing_accs.add(entry.id)
    return missing_accs
    
# file enthält alle kleiner accs die auch in der uniprot_species_nr DB drin waren
# nur 84 Identifikationen, 100% match: 3943 PSMs, #all matchs: 84 PSMs
acc_score_df = pd.read_csv('/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/number_ref_seq_in_db/matches_tax2proteome_kleiner_ref_one_accs.tsv', header=None, names=["acc", "score"], sep="\t")
acc_set_in_both = set(acc_score_df[acc_score_df.score == 100].acc)
# have to add sequences with no match
acc_set_only_in_ref = set(acc_score_df[acc_score_df.score != 100].acc)
all_acc_in_ref_df = {item for sublist in df_in_fdr_ref.Protein for item in sublist if 'REVERSED' not in item}
acc_no_blast_match = all_acc_in_ref_df.difference(set(acc_score_df.acc))
#print(acc_no_blast_match)
missing_acc_set = add_accs_with_no_diamond_result(ref_db, set(acc_score_df.acc))
print(len(missing_acc_set))
acc_set_only_in_ref = acc_set_only_in_ref.union(missing_acc_set)
print(len(acc_set_only_in_ref))
print(f"number sequences in tax2proteome uniprot species db (100% match): {len(acc_set_in_both)}\n\
number sequences only in reference: (<100 % match) {len(acc_set_only_in_ref)}\n\
number accs in ref DB: {len(acc_score_df)}")
ref_df_acc_not_in_t2p_df = df_in_fdr_ref[get_not_acc_rows(df_in_fdr_ref.Protein, acc_set_in_both, False)]
ref_df_acc_not_in_t2p_df_2 = df_in_fdr_ref[get_acc_rows(df_in_fdr_ref.Protein, acc_set_only_in_ref, True)]
ref_df_acc_in_t2p_df = df_in_fdr_ref[get_acc_rows(df_in_fdr_ref.Protein, acc_set_in_both, False)]
#print(ref_df_acc_not_in_t2p_df[['Title', 'Protein', 'Hyperscore']].head())
#print(ref_df_acc_not_in_t2p_df_2[['Title', 'Protein', 'Hyperscore']].head())
print(f"number identified spectra in ref in fdr: {len(set(df_in_fdr_ref.Title))}")
print(f"number identified spectra in ref in fdr with all acc in Protein set not 100% match to t2p DB: {len(set(ref_df_acc_not_in_t2p_df.Title))} \
together: {len(set(ref_df_acc_not_in_t2p_df.Title))+ len(set(ref_df_acc_in_t2p_df.Title))}")
print(f"number identified spectra in ref in fdr with all acc in Protein set not 100% match to t2p DB: {len(set(ref_df_acc_not_in_t2p_df_2.Title))} \
together: {len(set(ref_df_acc_not_in_t2p_df_2.Title))+ len(set(ref_df_acc_not_in_t2p_df_2.Title))}")
print(f"number identified spectra in T2P in fdr with acc also contained in ref: {len(set(ref_df_acc_in_t2p_df.Title))}")


In [ ]:
# 100% match: 3943 PSMs, #all matchs: 84 PSMs #unique spectra identified in Kleiner (>24): 564
# missing spectra with higher fdr:
df_uni_spec_in_missing_th = df_full_uniprot_species[(df_full_uniprot_species.Hyperscore >= 20.8)  & (df_full_uniprot_species.Hyperscore<23.7) & (df_full_uniprot_species.decoy!={True})].sort_values(by=['Hyperscore'])
spectra_in_this_range = set(df_uni_spec_in_missing_th.Title)
peptides_in_this_range = set(df_uni_spec_in_missing_th.Peptide)
df_spectra_in_this_range =df_in_fdr_uniprot_species[df_in_fdr_uniprot_species.Title.isin(spectra_in_this_range)]
df_peptides_in_this_range = df_in_fdr_uniprot_species[df_in_fdr_uniprot_species.Peptide.isin(peptides_in_this_range)]

print("number spectra in Hyperscore range betwenn 20.8 -23.7: ", len(spectra_in_this_range))
print("number peptides in Hyperscore range betwenn 20.8 -23.7: ", len(peptides_in_this_range))
print("len DF gefiltert nach Spektren, die in Hyperscore range betwenn 20.8 -23.7 liegen: "", len(df_spectra_in_this_range))
print("len DF gefiltert nach Peptiden, die in Hyperscore range betwenn 20.8 -23.7 liegen: ", len(df_peptides_in_this_range))
#print(peptides_in_this_range)
#print(df_uni_spec_in_missing_th.head())
#df_uni_spec_in_missing_th.tail()

In [ ]:

#proteins_ = {item for sublist in df_acc_not_in_this_range.Protein for item in sublist}
#print(len(proteins_.intersection(proteins_in_this_range)))
print('reference df: ', len(set(df_in_fdr_ref.Title)))
print('uniprot species df: ', len(set(df_in_fdr_uniprot_species.Title)))
print('uniprot species df acc: ', len(set(df_acc_not_in_this_range.Title)))
print('uniprot species df spectra: ',len(set(df_spectra_in_this_range.Title)))
print('uniprot species df peptides: ',len(set(df_peptides_not_in_this_range.Title)))
df_spectra_in_this_range

In [ ]:
# search peptides not in uniprot_nr species in DB
df_uni_spec_ref_th = df_full_uniprot_species[(df_full_uniprot_species.Hyperscore >= 20.8) & (df_full_uniprot_species.decoy!={True})]
#uniprot_identified_peptides_not_ref = set(df_uni_spec_ref_th.Peptide).difference(set(df_in_fdr_ref.Peptide))
ref_identified_peptides_not_in_t2p_uni_spec = set(df_in_fdr_ref.Peptide).difference(set(df_uni_spec_ref_th.Peptide))
print(len(ref_identified_peptides_not_in_t2p_uni_spec))
#print(len(uniprot_identified_peptides_not_ref))
#ref_identified_peptides_not_uniprot_species


In [ ]:
spectra_in_uni_set=set(df_in_fdr_uniprot_species.Title)
ref_df_spectra_not_in_t2p = df_in_fdr_ref[~df_in_fdr_ref.Title.isin(spectra_in_uni_set)].sort_values(by=['Hyperscore'])
acc_of_unidentified_spectra = {item for sublist in df_in_fdr_ref[~df_in_fdr_ref.Title.isin(spectra_in_uni_set)].Protein for item in sublist}
df_acc_not_in_species_df = df_in_fdr_ref[get_not_spectra_rows(df_in_fdr_ref.Title, spectra_in_uni_set)]
print(f"number spectra not identified with t2p DB: {len(set(ref_df_spectra_not_in_t2p.Title))}")
print(f"davon unter Hyperscore von 23.7 : {len(set((ref_df_spectra_not_in_t2p[ref_df_spectra_not_in_t2p.Hyperscore<23.7])))}")
print(f"number accs associate to these spectra: {len(acc_of_unidentified_spectra)}")
print(f"number of unidentified spectra with match < 100% with blast: {len(acc_of_unidentified_spectra.intersection(acc_set_only_in_ref))}")
print('Run1_U1_2000ng.74833.74833.2' in df_in_fdr_uniprot_species.Title)
print("VVALIEVADGEAAQPAAAAAPAPAK" in df_in_fdr_uniprot_species.Protein)
#print(df_in_fdr_ref[df_in_fdr_ref.Title == "Run1_U1_2000ng.74833.74833.2"][['Title','Hyperscore', 'Protein']])
#print(df_in_fdr_uniprot_species[df_in_fdr_uniprot_species.Title == "Run1_U1_2000ng.74833.74833.2"][['Title','Hyperscore', 'Protein']])
print(ref_df_spectra_not_in_t2p[['Hyperscore', 'Protein']].tail(5))

proteins_that_should_be_contained_in_t2P = {item for sublist in ref_df_spectra_not_in_t2p.Protein for item in sublist}.difference(acc_set_only_in_ref)
print(len(proteins_that_should_be_contained_in_t2P))
psm_that_should_be_contained_in_t2P = ref_df_spectra_not_in_t2p[get_acc_rows(ref_df_spectra_not_in_t2p.Protein, proteins_that_should_be_contained_in_t2P, True)]
psm_that_should_be_contained_in_t2P

In [ ]:
initial_set = set()
with open('/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/Tax2Proteome_DB/kleiner_species_nr.fasta', 'r') as input:
    for line in input:
        for pep in ref_identified_peptides_not_in_t2p_uni_spec:
            if pep in line:
                initial_set.add(pep)
print(len(initial_set))
#917 der in reference identifizierten Peptide, sind auch in t2p Datenbank enthalten

In [ ]:
uniprot_spec_db = list(SeqIO.parse("/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/Tax2Proteome_DB/kleiner_species_nr_crap.fasta", "fasta"))


In [ ]:
def search_db_for_peptide(db, peptide_set):
    new_dict= defaultdict(set)
    for entry in db:
        for peptide in peptide_set:
            if peptide in entry.seq:
                new_dict[peptide].add(entry.id)
    return new_dict
pep_to_acc_set_dict = search_db_for_peptide(uniprot_spec_db, set(psm_that_should_be_contained_in_t2P.Peptide))
print(len(pep_to_acc_set_dict))

In [ ]:
print(len(set(psm_that_should_be_contained_in_t2P.Peptide)))
list(pep_to_acc_set_dict.values())[0:2]
small_psm_uni = set(df_uni_spec_ref_th[df_uni_spec_ref_th.Hyperscore < 23.7].Title)
small_psm_ref = (set(psm_that_should_be_contained_in_t2P[psm_that_should_be_contained_in_t2P.Hyperscore < 23.7].Title))
print(len(small_psm_uni), len(small_psm_ref), len(small_psm_uni.intersection(small_psm_ref)))
all_spectra_in_ref_above_23_7 = set(df_in_fdr_ref[df_in_fdr_ref.Hyperscore > 23.7].Title)
print('in beiden identifizierte Spectra: ', len(set(df_in_fdr_uniprot_species.Title).intersection(all_spectra_in_ref_above_23_7)))
print('in ref identifizierte Spectra same th: ', len(all_spectra_in_ref_above_23_7))
print(36687-36119)
#568 spectren sind in ref über 23.7 aber nicht in t2p species
ref_identifiziert_nicht_in_t2p_df = df_in_fdr_ref[(df_in_fdr_ref.Hyperscore > 23.7) & \
                                                  (~ df_in_fdr_ref.Title.isin(set(df_in_fdr_uniprot_species.Title))) &\
                                                 (get_not_acc_rows(df_in_fdr_ref.Protein, acc_set_only_in_ref, False))]
proteins_not_identified = {item for sublist in ref_identifiziert_nicht_in_t2p_df.Protein for item in sublist}
peptides_not_identified = ref_identifiziert_nicht_in_t2p_df.Peptide
#pep_to_acc_set_dict = search_db_for_peptide(uniprot_spec_db, set(peptides_not_identified))
print(len(peptides_not_identified))
print("pep_to_acc_set_dict", len(pep_to_acc_set_dict))
print(len(proteins_not_identified))
print(len(set(ref_identifiziert_nicht_in_t2p_df.Title)))
from statistics import mean, median
evalues = {float(item[1:-1])  for item in ref_identifiziert_nicht_in_t2p_df.EValue}
print(median(evalues))

In [28]:
36687-36119

568

In [ ]:
d = search_db_for_peptide(uniprot_spec_db, {"PGEAPTTATSASPR"})
print(d['tr|Q1LRQ8|Q1LRQ8_CUPMC'])
# einige getestet, deren peptide sind wirklich in sequenz DB drin, wurden aber nicht identifiziert, keine Ahnung warum

In [ ]:
# anzahl der T2P unbekannten Peptide bei der Ref Identifizierung
ref_identifiziert_nicht_in_t2p_df = df_in_fdr_ref[(df_in_fdr_ref.Hyperscore > 23.7) & \
                                                  (~ df_in_fdr_ref.Title.isin(set(df_in_fdr_uniprot_species.Title))) &\
                                                 (get_acc_rows(df_in_fdr_ref.Protein, acc_set_only_in_ref, True))]
print(len(set(ref_identifiziert_nicht_in_t2p_df.Title)))
print(len(set(df_in_fdr_ref[(df_in_fdr_ref.Hyperscore < 23.7)].Title)))
print(len(set(df_in_fdr_ref[(df_in_fdr_ref.Hyperscore > 23.7) & \
                            (~ df_in_fdr_ref.Title.isin(set(df_in_fdr_uniprot_species.Title)))].Title)))
print(len(set(ref_identifiziert_nicht_in_t2p_df[ref_identifiziert_nicht_in_t2p_df.Peptide.isin(list(pep_to_acc_set_dict.keys()))].Title)))

In [ ]:
# Unterschiede ref identification vs. uniprot_species T2P identifications:
# #spectra in ref < 23.7 = 4542
# # spectra in ref identifiziert mittels t2p unbekannter seq: 392
# insgesamt # spectra nicht in t2b identifiziert: 4830, davon mit Hyperscore >23.7: 568
# bleiben 152 spectra, deren peptide auch in T2P drin waren, die haben einen mean Hyperscore von 27.1, Evalue: 0.36
# nicht identifizierte Peptide: 169, davon waren 144 in der T2P DB enthalten


In [ ]:
df_unidentified_uniprot = df_spectra_identified_exclusive_in_ref[get_acc_rows(df_spectra_identified_exclusive_in_ref.Protein, uni_accs)]